# Setting

设置全局的配置，主要包括mode、data_path等

In [27]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import time
import gc
import pickle
os.chdir("..") # set the root path 

In [4]:
!cat ./requirements.txt

tensorflow==1.14.0
gensim==3.8.3
faiss_cpu==1.6.3
lightgbm==2.3.1
deepctr==0.7.5
faiss==1.5.3
scikit_learn==0.23.1
pandas==0.20.1
numpy==1.15.4
tqdm==4.46.0

In [14]:
mode = 'online' # offline/online: offline validation or online submission
start_phase = 7
now_phase = 9

In [15]:
data_dir = 'data'
user_data_dir = 'user_data'

# online prediction data
online_train_path = os.path.join(data_dir, 'underexpose_train')
online_test_path = os.path.join(data_dir, 'underexpose_test')

# offline evaluation data
offline_train_path = os.path.join(user_data_dir, 'offline_underexpose_train')
offline_test_path = os.path.join(user_data_dir, 'offline_underexpose_test')
offline_answer_path = os.path.join(user_data_dir, 'offline_underexpose_answer')

train_file_prefix = 'underexpose_train_click'
test_file_prefix = 'underexpose_test_click'
infer_file_prefix = 'underexpose_test_qtime'
infer_answer_file_prefix = 'underexpose_test_qtime_with_answer'

item_feat_file_path = os.path.join(online_train_path, 'underexpose_item_feat.csv')
user_feat_file_path = os.path.join(online_train_path, 'underexpose_user_feat.csv')


# global variables to control online or offline
train_path = online_train_path if mode == 'online' else offline_train_path
test_path = online_test_path if mode == 'online' else offline_test_path

online_output_path = 'prediction_result'
offline_output_path = os.path.join(user_data_dir, 'prediction_result')

output_path = online_output_path if mode == 'online' else offline_output_path
if not os.path.exists(output_path): os.makedirs(output_path)

recommend_num = 800  # iterate number
topk_num = 200  # final recall number of each recall method

sr_gnn_root_dir = os.path.join(user_data_dir, 'sr-gnn', mode)
if not os.path.exists(sr_gnn_root_dir): os.makedirs(sr_gnn_root_dir)

## Data Split

验证集数据划分。做离线评估的时候，本方案主要在每个phase的点击数据，即all_click上分别进行。

从train数据集中随机采样1600个用户，将这1600个用户的最后一次交互item作为验证集answer，

其它数据作为验证集用户的历史行为数据。

In [16]:
sample_user_num = 1600
if not os.path.exists(offline_answer_path): os.makedirs(offline_answer_path)
if not os.path.exists(offline_test_path): os.makedirs(offline_test_path)
if not os.path.exists(offline_train_path): os.makedirs(offline_train_path)
np.random.seed(1234)  # reproduce-offline

def tr_val_split():
    for phase in range(now_phase + 1):
        click_train = pd.read_csv('{}/{}-{}.csv'.format(online_train_path, train_file_prefix, phase), header=None,
                                  names=['user_id', 'item_id', 'time'])
        all_user_ids = click_train['user_id'].unique()

        sample_user_ids = np.random.choice(all_user_ids, size=sample_user_num, replace=False)

        click_test = click_train[click_train['user_id'].isin(sample_user_ids)]
        click_train = click_train[~click_train['user_id'].isin(sample_user_ids)]

        click_test = click_test.sort_values(by=['user_id', 'time'])
        click_answer = click_test.groupby('user_id').tail(1)
        click_test = click_test.groupby('user_id').apply(lambda x: x[:-1]).reset_index(drop=True)
        click_answer = click_answer[click_answer['user_id'].isin(click_test['user_id'].unique())]  # 防止有些用户只有1个点击数据，去掉
        click_test = click_test[click_test['user_id'].isin(click_answer['user_id'].unique())]
        click_qtime = click_answer[['user_id', 'time']]

        click_train.to_csv(offline_train_path + '/{}-{}.csv'.format(train_file_prefix, phase), index=False, header=None)
        click_answer.to_csv(offline_answer_path + '/{}-{}.csv'.format(infer_answer_file_prefix, phase), index=False,
                            header=None)

        phase_test_path = "{}/{}-{}".format(offline_test_path, test_file_prefix, phase)
        if not os.path.exists(phase_test_path): os.makedirs(phase_test_path)
        click_test.to_csv(phase_test_path + '/{}-{}.csv'.format(test_file_prefix, phase), index=False, header=None)
        click_qtime.to_csv(phase_test_path + '/{}-{}.csv'.format(infer_file_prefix, phase), index=False, header=None)

In [5]:
tr_val_split()

# Recall
The code of this part is well checked, you can just run step by step.

## Common

### read item feat

In [29]:
from tqdm.notebook import tqdm
from collections import defaultdict  
import math  

item_feat_cols = ['item_id',] + ['txt_embed_'+ str(i) for i in range(128)] + ['img_embed_'+ str(i) for i in range(128)]
item_feat_df = pd.read_csv(item_feat_file_path, header=None, names=item_feat_cols)
item_feat_df['txt_embed_0'] = item_feat_df['txt_embed_0'].apply(lambda x:float(x[1:]))
item_feat_df['txt_embed_127'] = item_feat_df['txt_embed_127'].apply(lambda x:float(x[:-1]))
item_feat_df['img_embed_0'] = item_feat_df['img_embed_0'].apply(lambda x:float(x[1:]))
item_feat_df['img_embed_127'] = item_feat_df['img_embed_127'].apply(lambda x:float(x[:-1]))

### get topk-click items

In [32]:
# get online_topk
online_total_click = pd.DataFrame()
for c in range(now_phase + 1):
    print('phase:', c)
    click_train = pd.read_csv('{}/{}-{}.csv'.format(online_train_path, train_file_prefix, c), header=None,
                              names=['user_id', 'item_id', 'time'])
    
    phase_test_path = "{}/{}-{}".format(test_path, test_file_prefix, c)
    click_test = pd.read_csv('{}/{}-{}.csv'.format(phase_test_path, test_file_prefix, c), header=None,
                             names=['user_id', 'item_id', 'time'])

    all_click = click_train.append(click_test)
    all_click['phase'] = c
    online_total_click = online_total_click.append(all_click)
print(online_total_click.shape)
online_total_click = online_total_click.drop_duplicates(['user_id', 'item_id', 'time'])
print(online_total_click.shape)
online_top50_click_np = online_total_click['item_id'].value_counts().index[:50].values
online_top50_click = ','.join([str(i) for i in online_top50_click_np])

phase: 0
phase: 1
phase: 2
phase: 3
phase: 4
phase: 5
phase: 6
phase: 7
phase: 8
phase: 9
(2910846, 4)
(1263931, 4)


In [33]:
# get offline_topk
total_click = pd.DataFrame()
for c in range(now_phase + 1):
    print('phase:', c)
    click_train = pd.read_csv(train_path + '/underexpose_train_click-{}.csv'.format(c), header=None,
                              names=['user_id', 'item_id', 'time'])
    
    phase_test_path = "{}/{}-{}".format(offline_test_path, test_file_prefix, c)
    click_test = pd.read_csv(phase_test_path + '/{}-{}.csv'.format(test_file_prefix, c), header=None,
                              names=['user_id', 'item_id', 'time'])
    all_click = click_train.append(click_test)
    all_click['phase'] = c
    total_click = total_click.append(all_click)

print(total_click.shape)
total_click = total_click.drop_duplicates(['user_id', 'item_id', 'time'])
print(total_click.shape)
offline_top50_click_np = total_click['item_id'].value_counts().index[:50].values
offline_top50_click = ','.join([str(i) for i in offline_top50_click_np])

phase: 0
phase: 1
phase: 2
phase: 3
phase: 4
phase: 5
phase: 6
phase: 7
phase: 8
phase: 9
(2884240, 4)
(1247415, 4)


### obtain training data for each phase

In [34]:
def get_phase_click(c):
    '''
    get click data of target phase
    :param c: target phase
    :return: all_click (includes train and test), click_q_time (infer data, i.e., user_id q_time)
    '''
    print('train_path={}, test_path={}, target_phase={}'.format(train_path, test_path, c))

    click_train = pd.read_csv('{}/{}-{}.csv'.format(train_path, train_file_prefix, c), header=None,
                              names=['user_id', 'item_id', 'time'])

    phase_test_path = "{}/{}-{}".format(test_path, test_file_prefix, c)
    click_test = pd.read_csv('{}/{}-{}.csv'.format(phase_test_path, test_file_prefix, c), header=None,
                             names=['user_id', 'item_id', 'time'])

    phase_test_path = "{}/{}-{}".format(test_path, test_file_prefix, c)
    click_q_time = pd.read_csv('{}/{}-{}.csv'.format(phase_test_path, infer_file_prefix, c), header=None,
                               names=['user_id', 'time'])

    all_click = click_train.append(click_test)

    return all_click, click_q_time


def get_whole_phase_click(all_click, click_q_time):
    if mode == 'online':
        whole_click = online_total_click.drop_duplicates(['user_id', 'item_id', 'time'])
    else:
        whole_click = total_click.drop_duplicates(['user_id', 'item_id', 'time'])
        
    # items that occur in this phase
    phase_item_ids = set(all_click['item_id'].unique()) 
    
    pred_user_time_dict = dict(zip(click_q_time['user_id'], click_q_time['time']))
    
    def group_apply_func(group_df):
        u = group_df['user_id'].iloc[0]
        if u in pred_user_time_dict:
            u_time = pred_user_time_dict[u]
            group_df = group_df[group_df['time'] <= u_time]
        return group_df

    phase_whole_click = whole_click.groupby('user_id', group_keys=False).apply(group_apply_func)
    print(phase_whole_click.head())
    print('group done')
    
    # filter-out the items that not occur in the target phase
    phase_whole_click = phase_whole_click[phase_whole_click['item_id'].isin(phase_item_ids)]
    return phase_whole_click

def get_online_whole_click():
    '''
    get whole click
    :return: whole click data
    '''
    whole_click = pd.DataFrame()
    for c in range(now_phase + 1):
        click_train = pd.read_csv('{}/{}-{}.csv'.format(online_train_path, train_file_prefix, c), header=None,
                                  names=['user_id', 'item_id', 'time'])
        phase_test_path = "{}/{}-{}".format(online_test_path, test_file_prefix, c)
        click_test = pd.read_csv('{}/{}-{}.csv'.format(phase_test_path, test_file_prefix, c), header=None,
                                  names=['user_id', 'item_id', 'time'])
        all_click = click_train.append(click_test)
        all_click['phase'] = c
        whole_click = whole_click.append(all_click)

    whole_click = whole_click.drop_duplicates(['user_id', 'item_id', 'time'])
    return whole_click

### covert data format

In [35]:
def make_item_sim_tuple(group_df):
    group_df = group_df.sort_values(by=['sim'], ascending=False)
    item_score_tuples = list(zip(group_df['item_id'], group_df['sim']))
    return item_score_tuples


def save_recall_df_as_user_tuples_dict(total_recom_df, phase_full_sim_dict, prefix=''):
    save_path = os.path.join(user_data_dir, 'recall', mode)
    if not os.path.exists(save_path): os.makedirs(save_path)

    pickle.dump(total_recom_df, open(os.path.join(save_path, prefix + '_total_recall_df.pkl'), 'wb'))

    for phase in range(start_phase, now_phase + 1):
        phase_df = total_recom_df[total_recom_df['phase'] == phase]
        phase_user_item_score_dict = recall_df2dict(phase_df)
        phase_sim_dict = phase_full_sim_dict[phase]

        pickle.dump(phase_user_item_score_dict,
                    open(os.path.join(save_path, '{}_phase_{}.pkl'.format(prefix, phase)), 'wb'))
        pickle.dump(phase_sim_dict, open(os.path.join(save_path, '{}_phase_{}_sim.pkl'.format(prefix, phase)), 'wb'))


def sub2_df(filename):
    rec_items = []
    constant_sim = 100
    with open(filename) as f:
        for line in f:
            row = line.strip().split(",")
            uid = int(row[0])
            iids = row[1:]
            phase = uid % 11
            for idx, iid in enumerate(iids):
                rec_items.append((uid, int(iid), constant_sim - idx, phase))

    return pd.DataFrame(rec_items, columns=['user_id', 'item_id', 'sim', 'phase'])


def recall_df2dict(phase_df):
    phase_df = phase_df.groupby('user_id').apply(make_item_sim_tuple).reset_index().rename(
        columns={0: 'item_score_list'})
    item_score_list = phase_df['item_score_list'].apply(
        lambda item_score_list: sorted(item_score_list, key=lambda x: x[1], reverse=True))
    phase_user_item_score_dict = dict(zip(phase_df['user_id'], item_score_list))
    return phase_user_item_score_dict


def recall_dict2df(recall_item_score_dict):
    recom_list = []
    for u, item_score_list in recall_item_score_dict.items():
        for item, score in item_score_list:
            recom_list.append((u, item, score))
    return pd.DataFrame(recom_list, columns=['user_id', 'item_id', 'sim'])

### generate CF results

In [36]:
# fill user to 50 items  
def get_predict(df, pred_col, top_fill):
    top_fill = [int(t) for t in top_fill.split(',')]
    scores = [-1 * i for i in range(1, len(top_fill) + 1)]
    ids = list(df['user_id'].unique())
    fill_df = pd.DataFrame(ids * len(top_fill), columns=['user_id'])
    fill_df.sort_values('user_id', inplace=True)
    fill_df['item_id'] = top_fill * len(ids)
    fill_df[pred_col] = scores * len(ids)
    print(len(fill_df))
    df = df.append(fill_df)
    df.sort_values(pred_col, ascending=False, inplace=True)
    df = df.drop_duplicates(subset=['user_id', 'item_id'], keep='first')
    df['rank'] = df.groupby('user_id')[pred_col].rank(method='first', ascending=False)
    df = df[df['rank'] <= 50]
    df = df.groupby('user_id')['item_id'].apply(lambda x: ','.join([str(i) for i in x])).str.split(',',
                                                                                                   expand=True).reset_index()
    return df

In [37]:
from collections import defaultdict

def make_user_time_tuple(group_df, user_col='user_id', item_col='item_id', time_col='time'):
    user_time_tuples = list(zip(group_df[user_col], group_df[time_col]))
    return user_time_tuples

def make_item_time_tuple(group_df, user_col='user_id', item_col='item_id', time_col='time'):
   # group_df = group_df.drop_duplicates(subset=[user_col, item_col], keep='last')
    item_time_tuples = list(zip(group_df[item_col], group_df[time_col]))
    return item_time_tuples

def get_user_item_time_dict(df, user_col='user_id', item_col='item_id', time_col='time', is_drop_duplicated=False):
    user_item_ = df.sort_values(by=[user_col, time_col])
    
    if is_drop_duplicated:
        print('drop duplicates...')
        user_item_ = user_item_.drop_duplicates(subset=['user_id', 'item_id'], keep='last')
        
    user_item_ = user_item_.groupby(user_col).apply(lambda group: make_item_time_tuple(group, user_col, item_col, time_col)).reset_index().rename(columns={0: 'item_id_time_list'})
    user_item_time_dict = dict(zip(user_item_[user_col], user_item_['item_id_time_list']))
    return user_item_time_dict

def get_item_user_time_dict(df, user_col='user_id', item_col='item_id', time_col='time'):
    item_user_df = df.sort_values(by=[item_col, time_col])
    item_user_df = item_user_df.groupby(item_col).apply(
        lambda group: make_user_time_tuple(group, user_col, item_col, time_col)).reset_index().rename(
        columns={0: 'user_id_time_list'})
    item_user_time_dict = dict(zip(item_user_df[item_col], item_user_df['user_id_time_list']))
    return item_user_time_dict

def get_user_item_dict(df, user_col='user_id', item_col='item_id', time_col='time'):
    user_item_ = df.groupby(user_col)[item_col].agg(set).reset_index()
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))
    return user_item_dict

def get_user_min_time_dict(df,  user_col='user_id', item_col='item_id', time_col='time'):
    df = df.sort_values(by=[user_col, time_col])
    df = df.groupby(user_col).head(1)
    user_min_time_dict = dict(zip(df[user_col], df[time_col]))
    return user_min_time_dict


def item_based_recommend(sim_item_corr, user_item_time_dict, user_id, top_k, item_num, alpha=15000,
                         item_cnt_dict=None, user_cnt_dict=None, adjust_type='v2'):
    global item_content_sim_dict
    rank = {}
    if user_id not in user_item_time_dict:
        return []
    interacted_item_times = user_item_time_dict[user_id]
    min_time = min([time for item, time in interacted_item_times])
    interacted_items = set([item for item, time in interacted_item_times])

    miss_item_num = 0
    for loc, (i, time) in enumerate(interacted_item_times):
        if i not in sim_item_corr:
            miss_item_num += 1
            continue
        for j, wij in sorted(sim_item_corr[i].items(), key=lambda x: x[1], reverse=True)[0:top_k]:
            if j not in interacted_items:
                rank.setdefault(j, 0)

                content_weight = 1.0
                if item_content_sim_dict.get(i, {}).get(j, None) is not None:
                    content_weight += item_content_sim_dict[i][j]
                if item_content_sim_dict.get(j, {}).get(i, None) is not None:
                    content_weight += item_content_sim_dict[j][i]

                time_weight = np.exp(alpha * (time - min_time))
                loc_weight = (0.9 ** (len(interacted_item_times) - loc))
                rank[j] += loc_weight * time_weight * content_weight * wij
    if miss_item_num > 10:
        print('user_id={}, miss_item_num={}'.format(user_id, miss_item_num))

    if item_cnt_dict is not None:
        for loc, item in enumerate(rank):
            rank[item] = re_rank(rank[item], item, user_id, item_cnt_dict, user_cnt_dict, adjust_type=adjust_type)

    sorted_rank_items = sorted(rank.items(), key=lambda d: d[1], reverse=True)

    return sorted_rank_items[0:item_num]


def user_based_recommend(sim_user_corr, user_item_time_dict, user_id, top_k, item_num, alpha=15000,
                         item_cnt_dict=None, user_cnt_dict=None, adjust_type='v2'):
    global item_content_sim_dict

    rank = {}
    interacted_items = set([i for i, t in user_item_time_dict[user_id]])
    interacted_item_time_list = user_item_time_dict[user_id]
    interacted_num = len(interacted_items)

    min_time = min([t for i, t in interacted_item_time_list])
    time_weight_dict = {i: np.exp(alpha * (t - min_time)) for i, t in interacted_item_time_list}
    loc_weight_dict = {i: 0.9 ** (interacted_num - loc) for loc, (i, t) in enumerate(interacted_item_time_list)}

    for sim_v, wuv in sorted(sim_user_corr[user_id].items(), key=lambda x: x[1], reverse=True)[0:top_k]:
        if sim_v not in user_item_time_dict:
            continue
        for j, j_time in user_item_time_dict[sim_v]:
            if j not in interacted_items:
                rank.setdefault(j, 0)

                content_weight = 1.0
                for loc, (i, t) in enumerate(interacted_item_time_list):
                    loc_weight = loc_weight_dict[i]
                    time_weight = time_weight_dict[i]
                    if item_content_sim_dict.get(i, {}).get(j, None) is not None:
                        content_weight += time_weight * loc_weight * item_content_sim_dict[i][j]

                # weight = np.exp(-15000*abs(j_time-q_time))
                rank[j] += content_weight * wuv

    if item_cnt_dict is not None:
        for loc, item in enumerate(rank):
            rank[item] = re_rank(rank[item], item, user_id, item_cnt_dict, user_cnt_dict, adjust_type=adjust_type)

    rec_items = sorted(rank.items(), key=lambda d: d[1], reverse=True)

    return rec_items[:item_num]


def re_rank(sim, i, u, item_cnt_dict, user_cnt_dict, adjust_type='v2'):
    '''
    re_rank based on the popularity and similarity
    '''
    if adjust_type is None:
        return sim
    elif adjust_type == 'v1':
        # Log，Linear, 3/4
        if item_cnt_dict.get(i, 1.0) < 4:
            heat = np.log(item_cnt_dict.get(i, 1.0) + 2)
        elif item_cnt_dict.get(i, 1.0) >= 4 and item_cnt_dict.get(i, 1.0) < 10:
            heat = item_cnt_dict.get(i, 1.0)
        else:
            heat = item_cnt_dict.get(i, 1.0) ** 0.75 + 5.0  # 3/4
        sim *= 2.0 / heat

    elif adjust_type == 'v2':
        user_cnt = user_cnt_dict.get(u, 1.0)

        if item_cnt_dict.get(i, 1.0) < 4:
            heat = np.log(item_cnt_dict.get(i, 1.0) + 2)
        elif item_cnt_dict.get(i, 1.0) >= 4 and item_cnt_dict.get(i, 1.0) < 10:
            if user_cnt > 50:
                heat = item_cnt_dict.get(i, 1.0) * 1
            elif user_cnt > 25:
                heat = item_cnt_dict.get(i, 1.0) * 1.2
            else:
                heat = item_cnt_dict.get(i, 1.0) * 1.6
        else:
            if user_cnt > 50:
                user_cnt_k = 0.4
            elif user_cnt > 10:
                user_cnt_k = 0.1
            else:
                user_cnt_k = 0
            heat = item_cnt_dict.get(i, 1.0) ** user_cnt_k + 10 - 10 ** user_cnt_k  # 3/4
        sim *= 2.0 / heat

    else:
        sim += 2.0 / item_cnt_dict.get(i, 1.0)

    return sim

## Content-based

Reference:
    A library for efficient similarity search and clustering of dense vectors: https://github.com/facebookresearch/faiss


In [38]:
import collections
import pickle
import os

def get_content_sim_item(item_feat_df, topk=200, is_use_filled_feat=False, is_load_from_file=True):
    if not is_use_filled_feat:
        sim_path = os.path.join(user_data_dir, 'item_content_sim_dict.pkl')
    else:
        sim_path = os.path.join(user_data_dir, 'item_content_sim_dict_fill.pkl')

    if is_load_from_file and os.path.exists(sim_path):
        with open(sim_path, 'rb') as f:
            return pickle.load(f)
    print('begin compute similarity using faiss...')

    item_idx_2_rawid_dict = dict(zip(item_feat_df.index, item_feat_df['item_id']))
    txt_item_feat_df = item_feat_df.filter(regex="txt*")
    img_item_feat_df = item_feat_df.filter(regex="img*")

    txt_item_feat_np = np.ascontiguousarray(txt_item_feat_df.values, dtype=np.float32)
    img_item_feat_np = np.ascontiguousarray(img_item_feat_df.values, dtype=np.float32)

    # norm
    txt_item_feat_np = txt_item_feat_np / np.linalg.norm(txt_item_feat_np, axis=1, keepdims=True)
    img_item_feat_np = img_item_feat_np / np.linalg.norm(img_item_feat_np, axis=1, keepdims=True)

    txt_index = faiss.IndexFlatIP(128)
    txt_index.add(txt_item_feat_np)

    img_index = faiss.IndexFlatIP(128)
    img_index.add(img_item_feat_np)

    item_sim_dict = collections.defaultdict(dict)

    def search(feat_index, feat_np):
        sim, idx = feat_index.search(feat_np, topk)
        for target_idx, sim_value_list, rele_idx_list in zip(range(len(feat_np)), sim, idx):
            target_raw_id = item_idx_2_rawid_dict[target_idx]
            for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]):
                rele_raw_id = item_idx_2_rawid_dict[rele_idx]
                item_sim_dict[target_raw_id][rele_raw_id] = item_sim_dict.get(target_raw_id, {}).get(rele_raw_id,
                                                                                                     0) + sim_value

    search(txt_index, txt_item_feat_np)
    search(img_index, img_item_feat_np)

    if is_load_from_file:
        with open(sim_path, 'wb') as f:
            pickle.dump(item_sim_dict, f)

    return item_sim_dict

## Swing

In [39]:
def swing(df, user_col='user_id', item_col='item_id', time_col='time'):
    # 1. item, (u1,t1), (u2, t2).....
    item_user_df = df.sort_values(by=[item_col, time_col])
    item_user_df = item_user_df.groupby(item_col).apply(
        lambda group: make_user_time_tuple(group, user_col, item_col, time_col)).reset_index().rename(
        columns={0: 'user_id_time_list'})
    item_user_time_dict = dict(zip(item_user_df[item_col], item_user_df['user_id_time_list']))

    user_item_time_dict = defaultdict(list)
    # 2. ((u1, u2), i1, d12)
    u_u_cnt = defaultdict(list)
    item_cnt = defaultdict(int)
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        for u, u_time in user_time_list:
            # just record
            item_cnt[item] += 1
            user_item_time_dict[u].append((item, u_time))

            for relate_u, relate_u_time in user_time_list:
                if relate_u == u:
                    continue
               
                key = (u, relate_u)  if u <= relate_u else (relate_u, u)
                u_u_cnt[key].append((item, np.abs(u_time - relate_u_time)))


    # 3. (i1,i2), sim
    sim_item = {}
    alpha = 5.0
    for u_u, co_item_times in u_u_cnt.items():
        num_co_items = len(co_item_times)
        for i, i_time_diff in co_item_times:
            sim_item.setdefault(i, {})
            for j, j_time_diff in co_item_times:
              if j == i:
                continue
              weight = 1.0 # np.exp(-15000*(i_time_diff + j_time_diff)), not effective
              sim_item[i][j] = sim_item[i].setdefault(j, 0.) + weight / (alpha + num_co_items)
    # 4. norm by item count
    sim_item_corr = sim_item.copy()
    for i, related_items in sim_item.items():
        for j, cij in related_items.items():
            sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i] * item_cnt[j])
       
    return sim_item_corr, user_item_time_dict

## Item-CF

references: 
- A simple itemCF Baseline, score:0.1169(phase0-2): https://tianchi.aliyun.com/forum/postDetail?postId=103530
- 改进青禹小生baseline，phase3线上0.2: https://tianchi.aliyun.com/forum/postDetail?postId=105787

In [26]:
def get_time_dir_aware_sim_item(df, user_col='user_id', item_col='item_id', time_col='time'):
    user_item_time_dict = get_user_item_time_dict(df, user_col, item_col, time_col)

    sim_item = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        for loc_1, (i, i_time) in enumerate(item_time_list):
            item_cnt[i] += 1
            sim_item.setdefault(i, {})
            for loc_2, (relate_item, related_time) in enumerate(item_time_list):
                if i == relate_item:
                    continue
                loc_alpha = 1.0 if loc_2 > loc_1 else 0.7
                loc_weight = loc_alpha * (0.8**(np.abs(loc_2-loc_1)-1)) 
                time_weight = np.exp(-15000*np.abs(i_time-related_time))

                sim_item[i].setdefault(relate_item, 0)
                sim_item[i][relate_item] += loc_weight * time_weight / math.log(1 + len(item_time_list))
                
    sim_item_corr = sim_item.copy()
    for i, related_items in tqdm(sim_item.items()):
        for j, cij in related_items.items():
            sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i] * item_cnt[j])
            # sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i]*item_cnt[j])+cij/min(item_cnt[i], item_cnt[j])+0.5*cij/(item_cnt[i]+item_cnt[j])

    return sim_item_corr, user_item_time_dict


## Bi-Graph
References: A Simple Recall Method based on Network-based Inference, score:0.18 (phase0-3): https://tianchi.aliyun.com/forum/postDetail?postId=104936

In [27]:
def get_bi_sim_item(df, user_col='user_id', item_col='item_id', time_col='time'):
    item_user_time_dict = get_item_user_time_dict(df, user_col, item_col, time_col)
    user_item_time_dict = get_user_item_time_dict(df, user_col, item_col, time_col)

    item_cnt = defaultdict(int)  
    for user, item_times in tqdm(user_item_time_dict.items()):  
        for i,t in item_times:  
            item_cnt[i] += 1  

    sim_item = {}
    
    for item, user_time_lists in tqdm(item_user_time_dict.items()):
    
        sim_item.setdefault(item, {}) 
    
        for u, item_time in user_time_lists:
        
            tmp_len = len(user_item_time_dict[u])
        
            for relate_item, related_time in user_item_time_dict[u]:
                sim_item[item].setdefault(relate_item, 0)
                weight = np.exp(-15000*np.abs(related_time - item_time))
                sim_item[item][relate_item] += weight / (math.log(len(user_time_lists)+1) * math.log(tmp_len+1))
       
    return sim_item, user_item_time_dict

## User-CF

In [28]:
# user-cf
def get_sim_user(df, user_col='user_id', item_col='item_id', time_col='time'):
    # user_min_time_dict = get_user_min_time_dict(df, user_col, item_col, time_col) # user first time 
    # history
    user_item_time_dict = get_user_item_time_dict(df)
    # item, [u1, u2, ...,]
    item_user_time_dict = get_item_user_time_dict(df)

    sim_user = {}
    user_cnt = defaultdict(int)
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        num_users = len(user_time_list)
        for u, t in user_time_list:
            user_cnt[u] += 1
            sim_user.setdefault(u, {})
            for relate_user, relate_t in user_time_list:
                # time_diff_relate_u = 1.0/(1.0+10000*abs(relate_t-t))
                if u == relate_user:
                    continue
                sim_user[u].setdefault(relate_user, 0)
                weight = 1.0
                sim_user[u][relate_user] += weight / math.log(1 + num_users) # 流行度高的衰减

    sim_user_corr = sim_user.copy()
    for u, related_users in tqdm(sim_user.items()):
        for v, cuv in related_users.items():
            sim_user_corr[u][v] = cuv / math.sqrt(user_cnt[u] * user_cnt[v])

    return sim_user_corr, user_item_time_dict


## SR-GNN

we have put the results of the SR-GNN results in, 

- 'user_data/sr-gnn/online/{phase}/standard_rec.txt' 
- 'user_data/sr-gnn/online/{phase}/pos_node_weight_rec.txt'

So you can ignore the running of this part. If you want to try the GNN, you can run this part.

In [110]:
import re
import sys
sys.path.append('code/recall/sr_gnn/lib')
sr_gnn_lib_dir = 'code/recall/sr_gnn/lib'

### item content filling

**filling item features based on the local item-item co-occurrance**

In [69]:
txt_dense_feat = ['txt_embed_' + str(i) for i in range(128)]
img_dense_feat = ['img_embed_' + str(i) for i in range(128)]
item_dense_feat = txt_dense_feat + img_dense_feat

def process_item_feat(item_feat_df):
    processed_item_feat_df = item_feat_df.copy()
    # norm
    txt_item_feat_np = processed_item_feat_df[txt_dense_feat].values
    img_item_feat_np = processed_item_feat_df[img_dense_feat].values
    txt_item_feat_np = txt_item_feat_np / np.linalg.norm(txt_item_feat_np, axis=1, keepdims=True)
    img_item_feat_np = img_item_feat_np / np.linalg.norm(img_item_feat_np, axis=1, keepdims=True)
    processed_item_feat_df[txt_dense_feat] = pd.DataFrame(txt_item_feat_np, columns=txt_dense_feat)
    processed_item_feat_df[img_dense_feat] = pd.DataFrame(img_item_feat_np, columns=img_dense_feat)

    return processed_item_feat_df


def fill_item_feat(processed_item_feat_df, item_content_vec_dict):
    online_total_click = get_online_whole_click()

    all_click_feat_df = pd.merge(online_total_click, processed_item_feat_df, on='item_id', how='left')
    # miss items
    missed_items = all_click_feat_df[all_click_feat_df['txt_embed_0'].isnull()]['item_id'].unique()
    user_item_time_hist_dict = get_user_item_time_dict(online_total_click)

    # co-occurance
    co_occur_dict = {}
    window = 5

    def cal_occ(sentence):
        for i, word in enumerate(sentence):
            hist_len = len(sentence)
            co_occur_dict.setdefault(word, {})
            for j in range(max(i - window, 0), min(i + window, hist_len)):
                if j == i or word == sentence[j]: continue
                loc_weight = (0.9 ** abs(i - j))
                co_occur_dict[word].setdefault(sentence[j], 0)
                co_occur_dict[word][sentence[j]] += loc_weight

    for u, hist_item_times in user_item_time_hist_dict.items():
        hist_items = [i for i, t in hist_item_times]
        cal_occ(hist_items)

    # fill
    miss_item_content_vec_dict = {}
    for miss_item in missed_items:
        co_occur_item_dict = co_occur_dict[miss_item]
        weighted_vec = np.zeros(256)
        sum_weight = 0.0
        for co_item, weight in co_occur_item_dict.items():

            if co_item in item_content_vec_dict:
                sum_weight += weight
                co_item_vec = item_content_vec_dict[co_item]
                weighted_vec += weight * co_item_vec

        weighted_vec /= sum_weight
        txt_item_feat_np = weighted_vec[0:128] / np.linalg.norm(weighted_vec[0:128])
        img_item_feat_np = weighted_vec[128:] / np.linalg.norm(weighted_vec[128:])
        cnt_vec = np.concatenate([txt_item_feat_np, img_item_feat_np])
        miss_item_content_vec_dict[miss_item] = cnt_vec

    miss_item_feat_df = pd.DataFrame()
    miss_item_feat_df[item_dense_feat] = pd.DataFrame(miss_item_content_vec_dict.values(),
                                                      columns=item_dense_feat)
    miss_item_feat_df['item_id'] = list(miss_item_content_vec_dict.keys())
    miss_item_feat_df = miss_item_feat_df[['item_id'] + item_dense_feat]

    return miss_item_feat_df, miss_item_content_vec_dict


def obtain_entire_item_feat_df():
    global item_feat_df
    processed_item_feat_df = process_item_feat(item_feat_df)
    item_content_vec_dict = dict(zip(processed_item_feat_df['item_id'], processed_item_feat_df[item_dense_feat].values))
    miss_item_feat_df, miss_item_content_vec_dict = fill_item_feat(processed_item_feat_df, item_content_vec_dict)
    processed_item_feat_df = processed_item_feat_df.append(miss_item_feat_df)
    processed_item_feat_df = processed_item_feat_df.reset_index(drop=True)
    item_content_vec_dict.update(miss_item_content_vec_dict)
    return processed_item_feat_df, item_content_vec_dict

In [72]:
processed_item_feat_df, item_content_vec_dict = obtain_entire_item_feat_df()

### construct the training/validation data for SR-GNN

In [79]:
def construct_sr_gnn_train_data(target_phase, item_content_vec_dict, is_use_whole_click=True):

    # step 1: obtain original training data
    sr_gnn_dir = '{}/{}/data'.format(sr_gnn_root_dir, target_phase)
    if not os.path.exists(sr_gnn_dir): os.makedirs(sr_gnn_dir)
    all_click, click_q_time = get_phase_click(target_phase)
    phase_click = all_click
    if is_use_whole_click:
        print('using whole click to build training data')
        phase_whole_click = get_whole_phase_click(all_click, click_q_time)
        phase_click = phase_whole_click
    user_item_time_hist_dict = get_user_item_time_dict(phase_click)

    # step 2: encode the iid and uid
    # sparse features one-hot
    lbe = LabelEncoder()
    lbe.fit(phase_click['item_id'].astype(str))
    item_raw_id2_idx_dict = dict(zip(lbe.classes_,
                                     lbe.transform(lbe.classes_) + 1, ))  # 得到字典
    item_cnt = len(item_raw_id2_idx_dict)
    print(item_cnt)

    lbe = LabelEncoder()
    lbe.fit(phase_click['user_id'].astype(str))
    user_raw_id2_idx_dict = dict(zip(lbe.classes_,
                                     lbe.transform(lbe.classes_) + 1, ))  # dictionary
    user_cnt = len(user_raw_id2_idx_dict)
    print(user_cnt)

    # step 3: obtain feat to initialize embedding
    # step 3.1: item embedding
    item_embed_np = np.zeros((item_cnt + 1, 256))
    for raw_id, idx in item_raw_id2_idx_dict.items():
        vec = item_content_vec_dict[int(raw_id)]
        item_embed_np[idx, :] = vec
    np.save(open(sr_gnn_dir + '/item_embed_mat.npy', 'wb'), item_embed_np)
    # step 3.2: obtain user embedding (in fact, we don't use user embedding due to its limited performance)
    user_embed_np = np.zeros((user_cnt + 1, 256))
    for raw_id, idx in user_raw_id2_idx_dict.items():
        hist = user_item_time_hist_dict[int(raw_id)]
        vec = weighted_agg_content(hist, item_content_vec_dict)
        user_embed_np[idx, :] = vec
    np.save(open(sr_gnn_dir + '/user_embed_mat.npy', 'wb'), user_embed_np)

    # step 4: obtain item sequences based on the training data, i.e, train sequences, validate sequences, infer sequences
    full_user_item_dict = get_user_item_time_dict(phase_click)
    print(len(full_user_item_dict))
    # 4.1 train sequences
    train_user_hist_seq_dict = {}
    for u, hist_seq in full_user_item_dict.items():
        if len(hist_seq) > 1:
            train_user_hist_seq_dict[u] = hist_seq
    train_users = train_user_hist_seq_dict.keys()
    print(len(train_user_hist_seq_dict))
    # 4.2 validate sequences and infer sequences
    test_users = click_q_time['user_id'].unique()
    test_user_hist_seq_dict = {}
    infer_user_hist_seq_dict = {}
    for test_u in test_users:
        if test_u not in full_user_item_dict:
            print('test-user={} not in train/test data'.format(test_u))
            continue
        if len(full_user_item_dict[test_u]) > 1:
            test_user_hist_seq_dict[test_u] = full_user_item_dict[test_u]
            if test_u in train_user_hist_seq_dict:
                if len(train_user_hist_seq_dict[test_u][: -1]) > 1:
                    train_user_hist_seq_dict[test_u] = train_user_hist_seq_dict[test_u][: -1]  # last one not train, use just for test
                else:
                    del train_user_hist_seq_dict[test_u]

        infer_user_hist_seq_dict[test_u] = full_user_item_dict[test_u]

    print(len(train_user_hist_seq_dict))
    print(len(test_user_hist_seq_dict))
    print(len(infer_user_hist_seq_dict))

    # step 5: generate the data for the SR-GNN model
    def gen_data(is_attach_user=False):
        with open(sr_gnn_dir + '/train_item_seq.txt', 'w') as f_seq, open(sr_gnn_dir + '/train_user_sess.txt',
                                                                               'w') as f_user:
            for u in train_users:
                u_idx = user_raw_id2_idx_dict[str(u)]
                hist_item_time_seq = train_user_hist_seq_dict[u]
                hist_item_seq = [str(item_raw_id2_idx_dict[str(item)]) for item, time in hist_item_time_seq]
                if is_attach_user:
                    hist_item_seq_sess = [str(u_idx), ] + hist_item_seq
                else:
                    hist_item_seq_sess = hist_item_seq

                hist_item_seq_str = " ".join(hist_item_seq_sess)
                f_seq.write(hist_item_seq_str + '\n')

                # infer
                if is_attach_user:
                    hist_item_user_sess = [str(u), str(u_idx)] + hist_item_seq
                else:
                    hist_item_user_sess = [str(u), ] + hist_item_seq
                hist_item_user_sess_str = " ".join(hist_item_user_sess)
                f_user.write(hist_item_user_sess_str + '\n')

        with open(sr_gnn_dir + '/test_item_seq.txt', 'w') as f_seq, open(sr_gnn_dir + '/test_user_sess.txt',
                                                                              'w') as f_user:
            for u in test_users:
                # test
                if u in test_user_hist_seq_dict:
                    u_idx = user_raw_id2_idx_dict[str(u)]
                    hist_item_time_seq = test_user_hist_seq_dict[u]
                    hist_item_seq = [str(item_raw_id2_idx_dict[str(item)]) for item, time in hist_item_time_seq]

                    if is_attach_user:
                        hist_item_seq_sess = [str(u_idx), ] + hist_item_seq
                    else:
                        hist_item_seq_sess = hist_item_seq

                    hist_item_seq_str = " ".join(hist_item_seq_sess)
                    f_seq.write(hist_item_seq_str + '\n')

                if u in infer_user_hist_seq_dict:
                    hist_item_time_seq = infer_user_hist_seq_dict[u]
                    hist_item_seq = [str(item_raw_id2_idx_dict[str(item)]) for item, time in hist_item_time_seq]

                    if is_attach_user:
                        hist_item_user_sess = [str(u), str(u_idx)] + hist_item_seq
                    else:
                        hist_item_user_sess = [str(u), ] + hist_item_seq

                    hist_item_user_sess_str = " ".join(hist_item_user_sess)
                    f_user.write(hist_item_user_sess_str + '\n')

        with open(sr_gnn_dir + '/item_lookup.txt', 'w') as f_item_map:
            for raw_id, idx in item_raw_id2_idx_dict.items():
                f_item_map.write("{} {}\n".format(idx, raw_id))

    gen_data(is_attach_user=True)

     # step 6: enhance the data for the SR-GNN model to enrich the data. (convert the long sequences to multiple short sequences)
    def enhance_data(is_attach_user=False):
        np.random.seed(1234)
        count = 0
        max_len = 10
        tmp_max = 0
        with open(sr_gnn_dir + '/train_item_seq.txt', 'r') as f_in, open(
                sr_gnn_dir + '/train_item_seq_enhanced.txt', 'w') as f_out:
            for line in f_in:
                row = line.strip().split()

                if is_attach_user:
                    uid = row[0]
                    iids = row[1:]
                else:
                    iids = row

                end_step_1 = max(2, np.random.poisson(4))
                end_step_2 = len(iids) + 1

                if end_step_2 > end_step_1:
                    for i in range(end_step_1, end_step_2):
                        count += 1
                        start_end = max(i - max_len, 0)
                        tmp_max = max(tmp_max, len(iids[start_end: i]))
                        sampled_seq = iids[start_end: i]

                        if is_attach_user:
                            sampled_seq = [str(uid), ] + sampled_seq

                        f_out.write(' '.join(sampled_seq) + '\n')
                else:
                    count += 1
                    f_out.write(line)
        print("Done, Output Lines: {}".format(count))
        print(tmp_max)

    enhance_data(is_attach_user=True)

    return item_cnt # item_cnt just return as the input args for SR-GNN


def weighted_agg_content(hist_item_id_list, item_content_vec_dict):
    # weighted user behavior sequences to obtain user initial embedding
    weighted_vec = np.zeros(128*2)
    hist_num = len(hist_item_id_list)
    sum_weight = 0.0
    for loc, (i,t) in enumerate(hist_item_id_list):
        loc_weight = (0.9**(hist_num-loc))
        if i in item_content_vec_dict:
            sum_weight += loc_weight
            weighted_vec += loc_weight*item_content_vec_dict[i]
    if sum_weight != 0:
        weighted_vec /= sum_weight
        txt_item_feat_np = weighted_vec[0:128] / np.linalg.norm(weighted_vec[0:128])
        img_item_feat_np = weighted_vec[128:] / np.linalg.norm(weighted_vec[128:])
        weighted_vec = np.concatenate([txt_item_feat_np,  img_item_feat_np])
    else:
        print('zero weight...')
    return weighted_vec

In [80]:
phase_item_cnt_dict = {}
for phase in range(start_phase, now_phase+1):
    item_cnt = construct_sr_gnn_train_data(phase, item_content_vec_dict, is_use_whole_click=True)
    phase_item_cnt_dict[phase] = item_cnt
print('construct train data done...')

train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=7
using whole click to build training data
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742      0
20480        1    89568  0.983763      0
20968        1    97795  0.983877      0
group done
45194
34971
34971
33791
33756
1797
1797
Done, Output Lines: 651442
10
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=8
using whole click to build training data
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742      0
20480        1    89568  0.983763      0
20968        1    97795  0.983877      0
group done
44979
35012
35012
33819
33787
1818
1818
Done, Output Lines: 656631
10
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=9
using whole cli

In [81]:
phase_item_cnt_dict

{7: 45194, 8: 44979, 9: 44365}

### training and inferring

In [123]:
def find_checkpoint_path(phase, checkpoint_prefix='session_id', version='v1'):
    checkpoint_dir = 'tmp/model_saved/{}/{}/{}'.format(version, mode, phase)
    step_max = 0
    re_cp = re.compile("{}-(\d+)\.".format(checkpoint_prefix))
    for file in os.listdir(checkpoint_dir):
        so = re_cp.search(file)
        if so:
            step = int(so.group(1))
            step_max = step if step > step_max else step_max
    checkpoint_path = '{}/{}-{}'.format(checkpoint_dir, checkpoint_prefix, step_max)
    print('CheckPoint: {}'.format(checkpoint_path))
    return checkpoint_path

#### version 1: original paper realization + item embedding initialization + L2 norm

References:

[1] Wu S, Tang Y, Zhu Y, et al. Session-based recommendation with graph neural networks[C]//Proceedings of the AAAI Conference on Artificial Intelligence. 2019, 33: 346-353.

[2] Gupta P, Garg D, Malhotra P, et al. NISER: Normalized Item and Session Representations with Graph Neural Networks[J]. arXiv preprint arXiv:1909.04276, 2019.

phase 7: Best Recall and MRR: 0.1920,  0.0369  Epoch: 3,  4

In [113]:
# running model
for phase in range(start_phase, now_phase + 1):
    print('phase={}'.format(phase))
    model_path = 'tmp/model_saved/v1/{}/{}'.format(mode, phase)
    if not os.path.exists(model_path): os.makedirs(model_path)

    file_path = '{}/{}/data'.format(sr_gnn_root_dir, phase)
    if os.path.exists(model_path):
        print('model_path={} exists, delete'.format(model_path))
        os.system("rm -rf {}".format(model_path))
    item_cnt = phase_item_cnt_dict[phase]
    !python3 {sr_gnn_lib_dir}/main.py --task train --node_count {item_cnt} \
                --checkpoint_path {model_path}/session_id \
                --train_input {file_path}/train_item_seq_enhanced.txt \
                --test_input {file_path}/test_item_seq.txt --gru_step 2 \
                --epochs 10 --lr 0.001 --lr_dc 2 --dc_rate 0.1 --early_stop_epoch 3 \
                --hidden_size 256 --batch_size 256 --max_len 20 --has_uid True \
                --feature_init {file_path}/item_embed_mat.npy --sigma 8       

#     # generate rec
    checkpoint_path = find_checkpoint_path(phase, version='v1')
    prefix = 'standard_'

    rec_path = '{}/{}rec.txt'.format(file_path, prefix) # you can copy this to sr_gnn_root_dir/phase to be read by 2.7.4 method
    
    !python3 {sr_gnn_lib_dir}/main.py --task recommend --node_count {item_cnt} --checkpoint_path {checkpoint_path} \
              --item_lookup {file_path}/item_lookup.txt --recommend_output {rec_path} \
              --session_input {file_path}/test_user_sess.txt --gru_step 2 \
              --hidden_size 256 --batch_size 256 --rec_extra_count 50 --has_uid True \
              --feature_init {file_path}/item_embed_mat.npy \
              --max_len 10 --sigma 8

phase=7
model_path=tmp/model_saved/online/7 exists, delete

{'lr': 0.001, 'gru_step': 2, 'batch_size': 256, 'hidden_size': 256, 'epochs': 10, 'lr_dc': 2, 'dc_rate': 0.1, 'early_stop_epochs': 3, 'sigma': 8.0, 'max_len': 20, 'has_uid': True, 'feature_init': 'user_data/sr-gnn/online/7/data/item_embed_mat.npy', 'train_input': 'user_data/sr-gnn/online/7/data/train_item_seq_enhanced.txt', 'test_input': 'user_data/sr-gnn/online/7/data/test_item_seq.txt'}
2020-06-17 02:19:29,373 root:INFO:Data Loaded, Length: 651442， Max Length: 9
2020-06-17 02:19:29,391 root:INFO:Data Loaded, Length: 1797， Max Length: 19
2020-06-17 02:19:29,391 main:INFO:Train: {'lr': 0.001, 'gru_step': 2, 'batch_size': 256, 'hidden_size': 256, 'epochs': 10, 'lr_dc': 5090.0, 'dc_rate': 0.1, 'early_stop_epochs': 3, 'sigma': 8.0, 'max_len': 20, 'has_uid': True, 'feature_init': 'user_data/sr-gnn/online/7/data/item_embed_mat.npy', 'test_input': 'user_data/sr-gnn/online/7/data/test_item_seq.txt'}

2020-06-17 02:19:29,391 tensorflo

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-06-17 02:19:30,167 tensorflow:WARNING:From /home/researcher/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-06-17 02:19:30.814433: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-06-17 02:19:30.840430: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2500000000 Hz
2020-06-17 02:19:30.843787: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5695c90 executing computations on platform Host. Devices:
2020-06-17 02:19:30.843806: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecut

2020-06-17 02:59:14,097 main:INFO:Test Loss: 9.0409  @50, Recall: 0.1992  MRR: 0.0428
2020-06-17 02:59:15,210 main:INFO:Test Loss: 8.8047  @50, Recall: 0.2500  MRR: 0.0419
2020-06-17 02:59:16,314 main:INFO:Test Loss: 8.8804  @50, Recall: 0.1953  MRR: 0.0334
2020-06-17 02:59:17,417 main:INFO:Test Loss: 9.1498  @50, Recall: 0.1719  MRR: 0.0279
2020-06-17 02:59:18,527 main:INFO:Test Loss: 9.0279  @50, Recall: 0.1602  MRR: 0.0335
2020-06-17 02:59:19,636 main:INFO:Test Loss: 8.8918  @50, Recall: 0.1992  MRR: 0.0419
2020-06-17 02:59:20,743 main:INFO:Test Loss: 8.8552  @50, Recall: 0.1719  MRR: 0.0367
2020-06-17 02:59:20,788 main:INFO:Test Loss: 8.9623  @50, Recall: 0.0000  MRR: 0.0000
2020-06-17 02:59:20,789 main:INFO:Epoch: 3 Train Loss: 7.7835 Test Loss: 8.9516 Recall: 0.1920 MRR: 0.0368
2020-06-17 02:59:20,789 main:INFO:Best Recall and MRR: 0.1920,  0.0368  Epoch: 3,  3
2020-06-17 02:59:21,244 main:INFO:Total Batch: 2545
2020-06-17 02:59:21,440 main:INFO:Batch 0, Loss: 7.69111
2020-06-17 


2020-06-17 03:41:58,669 tensorflow:WARNING:From /home/researcher/workspace/MXDataMining/research/demo/kdd/B/KDD_CUP_2020_Debiasing_Rush/code/recall/sr_gnn/lib/modules/ggnn.py:11: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
2020-06-17 03:41:58,707 tensorflow:WARNING:From /home/researcher/workspace/MXDataMining/research/demo/kdd/B/KDD_CUP_2020_Debiasing_Rush/code/recall/sr_gnn/lib/modules/ggnn.py:16: GRUCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
2020-06-17 03:41:58,752 tensorflow:WARNING:From /home/researc

2020-06-17 03:42:00.028870: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-06-17 03:42:00.060514: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2500000000 Hz
2020-06-17 03:42:00.065069: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x42642e0 executing computations on platform Host. Devices:
2020-06-17 03:42:00.065118: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2020-06-17 03:42:00.854740: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAG

#### Version 2: version 1 + node weight + position embedding

phase 7: Best Recall and MRR: 0.2031,  0.0391  Epoch: 3,  5

In [125]:
# running model
for phase in [7]:
    print('phase={}'.format(phase))
    model_path = 'tmp/model_saved/v2/{}/{}'.format(mode, phase)
    if not os.path.exists(model_path): os.makedirs(model_path)

    file_path = '{}/{}/data'.format(sr_gnn_root_dir, phase)
    if os.path.exists(model_path):
        print('model_path={} exists, delete'.format(model_path))
        os.system("rm -rf {}".format(model_path))
    item_cnt = phase_item_cnt_dict[phase]
    
    !python3 {sr_gnn_lib_dir}/main.py --task train --node_count {item_cnt} \
              --checkpoint_path {model_path}/session_id --train_input {file_path}/train_item_seq_enhanced.txt \
              --test_input {file_path}/test_item_seq.txt --gru_step 2 --epochs 10 \
              --lr 0.001 --lr_dc 2 --dc_rate 0.1 --early_stop_epoch 3 --hidden_size 256 --batch_size 256 \
              --max_len 20 --has_uid True --feature_init {file_path}/item_embed_mat.npy --sigma 10 \
              --sq_max_len 5 --node_weight True  --node_weight_trainable True
    
    # generate rec
    checkpoint_path = find_checkpoint_path(phase, version='v2')
    prefix = 'pos_node_weight_'

    rec_path = '{}/{}rec.txt'.format(file_path, prefix)
    !python3 {sr_gnn_lib_dir}/main.py --task recommend --node_count {item_cnt} --checkpoint_path {checkpoint_path} \
              --item_lookup {file_path}/item_lookup.txt --recommend_output {rec_path} \
              --session_input {file_path}/test_user_sess.txt --gru_step 2 \
              --hidden_size 256 --batch_size 256 --rec_extra_count 50 --has_uid True \
              --feature_init {file_path}/item_embed_mat.npy \
              --max_len 10 --sigma 10 --sq_max_len 5 --node_weight True \
              --node_weight_trainable True

phase=7
model_path=tmp/model_saved/v2/online/7 exists, delete

{'lr': 0.001, 'gru_step': 2, 'batch_size': 256, 'hidden_size': 256, 'epochs': 10, 'lr_dc': 2, 'dc_rate': 0.1, 'early_stop_epochs': 3, 'sigma': 10.0, 'max_len': 20, 'has_uid': True, 'feature_init': 'user_data/sr-gnn/online/7/data/item_embed_mat.npy', 'node_weight': 'True', 'node_weight_trainable': True, 'sq_max_len': 5, 'train_input': 'user_data/sr-gnn/online/7/data/train_item_seq_enhanced.txt', 'test_input': 'user_data/sr-gnn/online/7/data/test_item_seq.txt'}
2020-06-17 04:07:05,116 root:INFO:Data Loaded, Length: 651442， Max Length: 9
2020-06-17 04:07:05,141 root:INFO:Data Loaded, Length: 1797， Max Length: 19
2020-06-17 04:07:24,441 main:INFO:Train: {'lr': 0.001, 'gru_step': 2, 'batch_size': 256, 'hidden_size': 256, 'epochs': 10, 'lr_dc': 5090.0, 'dc_rate': 0.1, 'early_stop_epochs': 3, 'sigma': 10.0, 'max_len': 20, 'has_uid': True, 'feature_init': 'user_data/sr-gnn/online/7/data/item_embed_mat.npy', 'node_weight': array([1.


2020-06-17 04:07:25,325 tensorflow:WARNING:From /home/researcher/workspace/MXDataMining/research/demo/kdd/B/KDD_CUP_2020_Debiasing_Rush/code/recall/sr_gnn/lib/model.py:58: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.


2020-06-17 04:07:25,330 tensorflow:WARNING:From /home/researcher/workspace/MXDataMining/research/demo/kdd/B/KDD_CUP_2020_Debiasing_Rush/code/recall/sr_gnn/lib/model.py:59: The name tf.trainable_variables is deprecated. Please use tf.compat.v1.trainable_variables instead.


2020-06-17 04:07:25,330 tensorflow:WARNING:From /home/researcher/workspace/MXDataMining/research/demo/kdd/B/KDD_CUP_2020_Debiasing_Rush/code/recall/sr_gnn/lib/model.py:60: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.


2020-06-17 04:07:25,352 tensorflow:WARNING:From /home/researcher/workspace/MXDataMining/research/demo/kdd/B/KDD_CUP_2020_Debiasing_Rush/code/recall/sr_gnn/lib/model.py:6

2020-06-17 04:33:19,388 main:INFO:Test Loss: 8.8394  @50, Recall: 0.0000  MRR: 0.0000
2020-06-17 04:33:19,389 main:INFO:Epoch: 1 Train Loss: 8.3278 Test Loss: 8.8343 Recall: 0.1970 MRR: 0.0374
2020-06-17 04:33:19,390 main:INFO:Best Recall and MRR: 0.1970,  0.0374  Epoch: 1,  1
2020-06-17 04:33:20,046 main:INFO:Total Batch: 2545
2020-06-17 04:33:20,332 main:INFO:Batch 0, Loss: 7.50224
2020-06-17 04:34:15,548 main:INFO:Batch 200, Loss: 7.51706
2020-06-17 04:35:10,532 main:INFO:Batch 400, Loss: 7.51861
2020-06-17 04:36:05,614 main:INFO:Batch 600, Loss: 7.52303
2020-06-17 04:37:00,690 main:INFO:Batch 800, Loss: 7.51661
2020-06-17 04:37:55,618 main:INFO:Batch 1000, Loss: 7.51952
2020-06-17 04:38:50,806 main:INFO:Batch 1200, Loss: 7.52025
2020-06-17 04:39:46,376 main:INFO:Batch 1400, Loss: 7.52285
2020-06-17 04:40:42,871 main:INFO:Batch 1600, Loss: 7.52492
2020-06-17 04:41:41,455 main:INFO:Batch 1800, Loss: 7.52750
2020-06-17 04:42:38,091 main:INFO:Batch 2000, Loss: 7.52979
2020-06-17 04:43:

2020-06-17 05:27:47,518 main:INFO:Batch 1400, Loss: 7.31963
2020-06-17 05:28:42,617 main:INFO:Batch 1600, Loss: 7.32102
2020-06-17 05:29:37,715 main:INFO:Batch 1800, Loss: 7.32108
2020-06-17 05:30:32,823 main:INFO:Batch 2000, Loss: 7.31977
2020-06-17 05:31:27,342 main:INFO:Batch 2200, Loss: 7.31903
2020-06-17 05:32:22,285 main:INFO:Batch 2400, Loss: 7.31968
2020-06-17 05:33:04,299 main:INFO:Test Loss: 8.9100  @50, Recall: 0.2070  MRR: 0.0518
2020-06-17 05:33:05,574 main:INFO:Test Loss: 8.6891  @50, Recall: 0.2617  MRR: 0.0401
2020-06-17 05:33:06,833 main:INFO:Test Loss: 8.8229  @50, Recall: 0.1953  MRR: 0.0361
2020-06-17 05:33:08,106 main:INFO:Test Loss: 9.0597  @50, Recall: 0.1836  MRR: 0.0291
2020-06-17 05:33:09,376 main:INFO:Test Loss: 8.9356  @50, Recall: 0.1602  MRR: 0.0367
2020-06-17 05:33:10,638 main:INFO:Test Loss: 8.7341  @50, Recall: 0.2188  MRR: 0.0415
2020-06-17 05:33:11,916 main:INFO:Test Loss: 8.7610  @50, Recall: 0.1719  MRR: 0.0383
2020-06-17 05:33:11,971 main:INFO:Test

2020-06-17 05:57:29,611 tensorflow:WARNING:Entity <bound method GRUCell.call of <tensorflow.python.ops.rnn_cell_impl.GRUCell object at 0x7f964ad71be0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method GRUCell.call of <tensorflow.python.ops.rnn_cell_impl.GRUCell object at 0x7f964ad71be0>>: AssertionError: Bad argument number for Name: 3, expecting 4
2020-06-17 05:57:29,822 tensorflow:WARNING:Entity <bound method GRUCell.call of <tensorflow.python.ops.rnn_cell_impl.GRUCell object at 0x7f964ad71be0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method GRUCell.call of <tensorflow.python.ops.rnn_cell_impl.GRU

### read the output results of SR-GNN

In [31]:
def filter_df(recom_df, phase, is_item_cnt_weight=False, adjust_type='v2'):
    print(len(recom_df))
    filter_num = 0

    all_click, click_q_time = get_phase_click(phase)
    phase_whole_click = get_whole_phase_click(all_click, click_q_time)

    if mode == 'online':
        user_item_hist_dict = get_user_item_dict(phase_whole_click)
    else:
        user_item_hist_dict = get_user_item_dict(all_click)

    item_cnt_dict = all_click.groupby('item_id')['user_id'].count().to_dict()
    user_cnt_dict = all_click.groupby('user_id')['item_id'].count().to_dict()

    recom_list = []
    for row in recom_df.itertuples(index=False):
        uid = int(row.user_id)
        iid = int(row.item_id)
        if uid in user_item_hist_dict and iid in user_item_hist_dict[uid]:
            filter_num += 1
            continue
        sim = row.sim
        if is_item_cnt_weight:
            sim = re_rank(row.sim, iid, uid, item_cnt_dict, user_cnt_dict, adjust_type=adjust_type)
        #             sim = row.sim * 2.0 / item_cnt_dict.get(iid, 1.0)
        recom_list.append((uid, iid, sim, row.phase))

    print('num={}, filter_num={}'.format(len(recom_list), filter_num))
    filter_recom_df = pd.DataFrame(recom_list, columns=['user_id', 'item_id', 'sim', 'phase'])
    return filter_recom_df


def read_sr_gnn_results(phase, prefix='standard', adjust_type='v2'):
    print('sr-gnn begin...')
    sr_gnn_rec_path = '{}/{}/{}_rec.txt'.format(sr_gnn_root_dir, phase, prefix)  # standard_rec.txt + pos_node_weight_rec.txt
    rec_user_item_dict = {}
    with open(sr_gnn_rec_path) as f:
        for line in f:
            try:
                row = eval(line)
                uid = row[0]
                iids = row[1]
                iids = [(int(iid), float(score)) for iid, score in iids]
                iids = sorted(iids, key=lambda x: x[1], reverse=True)
                rec_user_item_dict[int(uid)] = iids
            except:
                print(line)
    print('read sr-gnn done, num={}'.format(len(rec_user_item_dict)))
    recom_df = recall_dict2df(rec_user_item_dict)
    recom_df['phase'] = phase
    recom_df = filter_df(recom_df, phase, is_item_cnt_weight=True, adjust_type=adjust_type)
    recall_user_item_score_dict = recall_df2dict(recom_df)
    return recall_user_item_score_dict

In [147]:
# observe the results
sr_gnn_recall_user_item_score_dict = read_sr_gnn_results(7, prefix='standard', adjust_type='v2')

sr-gnn begin...
read sr-gnn done, num=1797
179700
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=7
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742      0
20480        1    89568  0.983763      0
20968        1    97795  0.983877      0
group done
num=176238, filter_num=3462


In [152]:
# observe
get_predict(recall_dict2df(sr_gnn_recall_user_item_score_dict), 'sim', online_top50_click)

89850


user_id       0       1       2       3       4       5       6       7  \
0           7  115146  117519  115074  114953  117520  116220  117518  116475   
1          29   16401   60886   56669   26442   14038   16300   25868   39785   
2          40    6078    1239     729    1951   43338   10881   43044    6502   
3          51   12824  114201   88107   21957   24974   96907   31297   12361   
4          73   50592   25540   51143   12899    6935   10580   70124   15860   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
1792    35328   38691  107596   53829   99500   68566   49582   59009   12408   
1793    35339  102105   37743   13577    8198   39016    8180    4483   57580   
1794    35361   60544    1609     771    1667     776     951   34083     271   
1795    35383   12638   37431   40959   42897   15268   90067    2064   14799   
1796    35394   95672   33410  106301   97332    9978   18155   58340   61453   

           8  ...      40      41     42      43      44      45     46  \
0     117545  ...   91297  112296  12661   13954   27455  116327  58934   
1      93446  ...   26380   16977  76710   45767   68976   27921  71497   
2       9075  ...   45687    4492  64416    6151    3914    8461   8481   
3      89040  ...   97873   24488  88605   44841   89851   67903  91012   
4      35758  ...    6647   71613  36187   37134   21129   71628  80295   
...      ...  ...     ...     ...    ...     ...     ...     ...    ...   
1792   69993  ...   33698  103045  59045   96988   50024   64551  49975   
1793    2545  ...    6241    3192   7795   55873  113550    4647  12757   
1794     778  ...    3120   27648  65355  103730   74261    4609  51955   
1795   60174  ...   29520   59037  66353   32466   91687   21943  47461   
1796   40824  ...  104649   24834  54874   42519   91945  103797  89466   

          47      48      49  
0     116980  115162  115147  
1      22637  115735   27872  
2      79820     299   19882  
3     100727   81986  102924  
4      52670   41902   72164  
...      ...     ...     ...  
1792   75400   37939   54708  
1793    4230    4136    3767  
1794    4454   29953   30354  
1795   40280  103947   19517  
1796   28930   34518   47937  

[1797 rows x 51 columns]

## recall process


### recall one source

In [45]:
def norm_recall_item_score_list(sorted_recall_item_list):
    if len(sorted_recall_item_list) == 0: return sorted_recall_item_list

    assert sorted_recall_item_list[0][1] >= sorted_recall_item_list[-1][1]  # check whether it's the ranked result
    max_sim = sorted_recall_item_list[0][1]
    min_sim = sorted_recall_item_list[-1][1]

    norm_sorted_recall_item_list = []
    for item, score in sorted_recall_item_list:
        if max_sim > 0:
            norm_score = 1.0 * (score - min_sim) / (max_sim - min_sim) if max_sim > min_sim else 1.0
        else:
            norm_score = 0.0  # topk-fill set to 0.0
        norm_sorted_recall_item_list.append((item, norm_score))
    return norm_sorted_recall_item_list


def norm_user_recall_item_dict(recall_item_dict):
    norm_recall_item_dict = {}
    for u, sorted_recall_item_list in recall_item_dict.items():
        norm_recall_item_dict[u] = norm_recall_item_score_list(sorted_recall_item_list)
    return norm_recall_item_dict


def get_recall_results(item_sim_dict, user_item_dict, target_user_ids=None, item_based=True,
                       item_cnt_dict=None, user_cnt_dict=None, adjust_type='xtf_v6'):
    if target_user_ids is None:
        target_user_ids = user_item_dict.keys()
    recall_item_dict = {}

    if mode == 'online':
        top50_click_np = online_top50_click_np
    else:
        top50_click_np = offline_top50_click_np

    print('adjust_type={}'.format(adjust_type))

    for u in tqdm(target_user_ids):
        if item_based:
            recall_items = item_based_recommend(item_sim_dict, user_item_dict, u, recommend_num, topk_num,
                                                item_cnt_dict=item_cnt_dict, user_cnt_dict=user_cnt_dict,
                                                adjust_type=adjust_type)
        else:
            recall_items = user_based_recommend(item_sim_dict, user_item_dict, u, recommend_num, topk_num,
                                                item_cnt_dict=item_cnt_dict, user_cnt_dict=user_cnt_dict,
                                                adjust_type=adjust_type)

        if len(recall_items) == 0:
            recall_items = [(top50_click_np[0], 0.0)]  # to avoid the lost of the recommendation results for this user

        recall_item_dict[u] = recall_items

    return recall_item_dict

### aggregate multi-recall sources

In [34]:
# item-cf
# bi-graph
# user-cf
# item-cf
def agg_recall_results(recall_item_dict_list_dict, is_norm=True, ret_type='tuple',
                       weight_dict={}):
    print('aggregate recall results begin....')
    agg_recall_item_dict = {}
    for name, recall_item_dict in recall_item_dict_list_dict.items():
        if is_norm:
            recall_item_dict = norm_user_recall_item_dict(recall_item_dict)
        weight = weight_dict.get(name, 1.0)
        print('name={}, weight={}'.format(name, weight))
        for u, recall_items in recall_item_dict.items():
            agg_recall_item_dict.setdefault(u, {})
            for i, score in recall_items:
                agg_recall_item_dict[u].setdefault(i, 0.0)
                agg_recall_item_dict[u][i] += weight * score  # 累加

    if ret_type == 'tuple':
        agg_recall_item_tuple_dict = {}
        for u, recall_item_dict in agg_recall_item_dict.items():
            sorted_recall_item_tuples = sorted(recall_item_dict.items(), key=lambda x: x[1], reverse=True)
            agg_recall_item_tuple_dict[u] = sorted_recall_item_tuples
        return agg_recall_item_tuple_dict

    if ret_type == 'df':
        recall_u_i_score_pair_list = []
        for u, recall_item_dict in agg_recall_item_dict.items():
            for i, score in recall_item_dict.items():
                recall_u_i_score_pair_list.append((u, i, score))
        recall_df = pd.DataFrame.from_records(recall_u_i_score_pair_list, columns=['user_id', 'item_id', 'sim'])
        return recall_df

    return agg_recall_item_dict

### do recall 

**single thread version, may consume more time**

In [139]:
def get_multi_source_sim_dict_results(history_df, recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}):
    recall_sim_pair_dict = {}
    if 'item-cf' in recall_methods:
        print('item-cf item-sim begin')
        item_sim_dict, _ = get_time_dir_aware_sim_item(history_df)
        recall_sim_pair_dict['item-cf'] = item_sim_dict
        print('item-cf item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

    if 'bi-graph' in recall_methods:
        print('bi-graph item-sim begin')
        item_sim_dict, _ = get_bi_sim_item(history_df)
        recall_sim_pair_dict['bi-graph'] = item_sim_dict
        print('bi-graph item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

    if 'swing' in recall_methods:
        print('swing item-sim begin')
        item_sim_dict, _ = swing(history_df)
        recall_sim_pair_dict['swing'] = item_sim_dict
        print('swing item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

    if 'user-cf' in recall_methods:
        print('user-cf user-sim begin')
        user_sim_dict, _ = get_sim_user(history_df)
        recall_sim_pair_dict['user-cf'] = user_sim_dict
        print('user-cf user-sim-pair done, pair_num={}'.format(len(user_sim_dict)))

    return recall_sim_pair_dict


def do_multi_recall_results(recall_sim_pair_dict, user_item_time_dict,
                            target_user_ids=None, ret_type='df',
                            item_cnt_dict=None, user_cnt_dict=None, adjust_type='v2'):
    if target_user_ids is None:
        target_user_ids = user_item_time_dict.keys()

    recall_item_list_dict = {}
    for name, sim_dict in recall_sim_pair_dict.items():
        # item-based
        if name in {'item-cf', 'bi-graph', 'swing'}:
            recall_item_dict = get_recall_results(sim_dict, user_item_time_dict, target_user_ids, item_based=True,
                                                  item_cnt_dict=item_cnt_dict, user_cnt_dict=user_cnt_dict,
                                                  adjust_type=adjust_type)
        else:
            recall_item_dict = get_recall_results(sim_dict, user_item_time_dict, target_user_ids, item_based=False,
                                                  item_cnt_dict=item_cnt_dict, user_cnt_dict=user_cnt_dict,
                                                  adjust_type=adjust_type)

        print('{} recall done, recall_user_num={}.'.format(name, len(recall_item_dict)))
        recall_item_list_dict[name] = recall_item_dict

    standard_sr_gnn_recall_item_dict = read_sr_gnn_results(phase, prefix='standard',
                                                           adjust_type=adjust_type)
    pos_weight_sr_gnn_recall_item_dict = read_sr_gnn_results(phase, prefix='pos_node_weight',
                                                             adjust_type=adjust_type)

    recall_item_list_dict['sr_gnn_feat_init_v1'] = standard_sr_gnn_recall_item_dict
    recall_item_list_dict['sr_gnn_pos_weight_v2'] = pos_weight_sr_gnn_recall_item_dict

    return agg_recall_results(recall_item_list_dict, is_norm=True, ret_type=ret_type)

### do recall multi-processing 

**if your machine has more than 32G memorys, you can run this multi-processing version**

In [138]:
from multiprocessing import Process, JoinableQueue, Queue

def get_multi_source_sim_dict_results_multi_processing(history_df,
                                                       recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}):
    def convert(history_df, input_q, result_q):
        while True:
            name = input_q.get()
            if 'item-cf' == name:
                print('item-cf item-sim begin')
                item_sim_dict, _ = get_time_dir_aware_sim_item(history_df)
                result_q.put((name, item_sim_dict))
                print('item-cf item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

            elif 'bi-graph' == name:
                print('bi-graph item-sim begin')
                item_sim_dict, _ = get_bi_sim_item(history_df)
                result_q.put((name, item_sim_dict))
                print('bi-graph item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

            elif 'swing' == name:
                print('swing item-sim begin')
                item_sim_dict, _ = swing(history_df)
                result_q.put((name, item_sim_dict))
                print('swing item-sim-pair done, pair_num={}'.format(len(item_sim_dict)))

            elif 'user-cf' == name:
                print('user-cf user-sim begin')
                user_sim_dict, _ = get_sim_user(history_df)
                result_q.put((name, user_sim_dict))
                print('user-cf user-sim-pair done, pair_num={}'.format(len(user_sim_dict)))
            input_q.task_done()

    input_q = JoinableQueue()
    result_q = Queue()

    processes = []
    for name in recall_methods:
        input_q.put(name)
        processes.append(Process(target=convert, args=(history_df, input_q, result_q)))
        processes[-1].daemon = True
        processes[-1].start()

    input_q.join()

    recall_sim_pair_dict = {}
    while len(recall_sim_pair_dict) != len(recall_methods):
        print('current_len={}'.format(len(recall_sim_pair_dict)))
        if len(recall_sim_pair_dict) == len(recall_methods):
            break
        name, sim_pair_dict = result_q.get()
        recall_sim_pair_dict[name] = sim_pair_dict
    for p in processes:
        p.terminate()
        p.join()

    assert len(recall_sim_pair_dict) == len(recall_methods)
    return recall_sim_pair_dict


def do_multi_recall_results_multi_processing(recall_sim_pair_dict, user_item_time_dict, target_user_ids=None,
                                             ret_type='df', item_cnt_dict=None, user_cnt_dict=None, phase=None, adjust_type='v2',
                                             recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing', 'sr-gnn'}):
    from multiprocessing import Process, JoinableQueue, Queue

    print('recall-source-num={}'.format(len(recall_sim_pair_dict)))

    def convert(user_item_time_dict, target_user_ids, item_based, input_q, result_q):
        while True:
            name, sim_dict = input_q.get()
            print('do recall for {}'.format(name))
            recall_item_dict = get_recall_results(sim_dict, user_item_time_dict, target_user_ids, item_based=item_based,
                                                  item_cnt_dict=item_cnt_dict, user_cnt_dict=user_cnt_dict,
                                                  adjust_type=adjust_type)
            result_q.put((name, recall_item_dict))
            print('{} recall done, recall_user_num={}.'.format(name, len(recall_item_dict)))
            input_q.task_done()

    input_q = JoinableQueue()
    result_q = Queue()

    if target_user_ids is None:
        target_user_ids = user_item_time_dict.keys()

    processes = []
    for name, sim_dict in recall_sim_pair_dict.items():
        item_based = True if name in {'item-cf', 'bi-graph', 'swing'} else False
        input_q.put((name, sim_dict))
        processes.append(
            Process(target=convert, args=(user_item_time_dict, target_user_ids, item_based, input_q, result_q)))
        processes[-1].daemon = True
        processes[-1].start()

    input_q.join()

    recall_item_dict_list_dict = {}
    while len(recall_item_dict_list_dict) != len(recall_sim_pair_dict):
        print('current_len={}'.format(len(recall_item_dict_list_dict)))
        if len(recall_item_dict_list_dict) == len(recall_sim_pair_dict):
            break
        name, recall_item_dict = result_q.get()
        recall_item_dict_list_dict[name] = recall_item_dict

    for p in processes:
        p.terminate()
        p.join()

    print(len(recall_item_dict_list_dict))

    assert len(recall_item_dict_list_dict) == len(recall_sim_pair_dict)

    if 'sr-gnn' in recall_methods:
        print('read sr-gnn results....')
        standard_sr_gnn_recall_item_dict = read_sr_gnn_results(phase, prefix='standard', adjust_type=adjust_type)
        recall_item_dict_list_dict['sr_gnn_feat_init_v1'] = standard_sr_gnn_recall_item_dict
        print('read standard sr-gnn results done....')
        pos_weight_sr_gnn_recall_item_dict = read_sr_gnn_results(phase, prefix='pos_node_weight',
                                                                 adjust_type=adjust_type)
        recall_item_dict_list_dict['sr_gnn_pos_weight_v2'] = pos_weight_sr_gnn_recall_item_dict
        print('read pos_weight sr-gnn results done....')

    return agg_recall_results(recall_item_dict_list_dict, is_norm=True, ret_type=ret_type)


## recall-submit running

In [74]:
# obtain using faiss
item_content_sim_dict = get_content_sim_item(item_feat_df, topk=200)
len(item_content_sim_dict)

108916

### all_click

**offline evaluation using this version**

In [85]:
if mode == 'offline':
    recom_item = []
    print("train_path={}, test_path={}".format(train_path, test_path))
    whole_click = pd.DataFrame()

    total_recom_df = pd.DataFrame()
    for c in range(start_phase, now_phase + 1):
        print('phase:', c)
        all_click, click_q_time = get_phase_click(c)
        item_cnt_dict = all_click.groupby('item_id')['user_id'].count().to_dict()
        user_cnt_dict = all_click.groupby('user_id')['item_id'].count().to_dict()

        recall_methods={'swing', 'bi-graph', 'user-cf', 'swing'}
        recall_sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(all_click, 
                                                                                  recall_methods=recall_methods)
        user_item_time_dict = get_user_item_time_dict(all_click)

        recom_df = do_multi_recall_results_multi_processing(recall_sim_pair_dict, user_item_time_dict,
                                                            target_user_ids=click_q_time['user_id'].unique(), ret_type='df',
                                                            item_cnt_dict=item_cnt_dict, user_cnt_dict=user_cnt_dict,
                                                            phase=c, adjust_type='v2',
                                                            recall_methods=recall_methods)
        recom_df['phase'] = c
        total_recom_df = total_recom_df.append(recom_df)

    result = get_predict(total_recom_df, 'sim', offline_top50_click)
    result.to_csv(output_path + '/baseline_full_cf.csv', index=False, header=None)

### phase_whole_click

**online submission using this version**

In [53]:
total_recom_df = pd.DataFrame()
phase_full_sim_dict = {}

recall_methods = {'item-cf', 'bi-graph', 'swing', 'user-cf'}

for c in range(start_phase, now_phase + 1):
    print('phase:', c)
    all_click, click_q_time = get_phase_click(c)
    phase_whole_click = get_whole_phase_click(all_click, click_q_time)
    item_cnt_dict = all_click.groupby('item_id')['user_id'].count().to_dict()
    user_cnt_dict = all_click.groupby('user_id')['item_id'].count().to_dict()

    recall_sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(phase_whole_click,
                                                                              recall_methods=recall_methods)

    user_item_time_dict = get_user_item_time_dict(phase_whole_click, is_drop_duplicated=True)

    recom_df = do_multi_recall_results_multi_processing(recall_sim_pair_dict, user_item_time_dict,
                                                        target_user_ids=click_q_time['user_id'].unique(), ret_type='df',
                                                        item_cnt_dict=item_cnt_dict, user_cnt_dict=user_cnt_dict,
                                                        phase=c, adjust_type='v2',
                                                        recall_methods=recall_methods | {'sr-gnn'})
    recom_df['phase'] = c
    total_recom_df = total_recom_df.append(recom_df)
    phase_full_sim_dict[c] = recall_sim_pair_dict
    
# save for ranking         
today = time.strftime("%Y%m%d")
save_recall_df_as_user_tuples_dict(total_recom_df, phase_full_sim_dict,
                                       prefix='B-recall-{}'.format(today))

result = get_predict(total_recom_df, 'sim', online_top50_click)
result.to_csv(output_path + '/result_notebook.csv', index=False, header=None)

phase: 7
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=7
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742      0
20480        1    89568  0.983763      0
20968        1    97795  0.983877      0
group done
swing item-sim begin
bi-graph item-sim begin
item-cf item-sim begin
user-cf user-sim begin



user-cf user-sim-pair done, pair_num=34971

bi-graph item-sim-pair done, pair_num=45194


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




bi-graph recall done, recall_user_num=1797.

item-cf recall done, recall_user_num=1797.

user-cf recall done, recall_user_num=1797.
current_len=0
current_len=1
current_len=2
current_len=3
4
read sr-gnn results....
sr-gnn begin...
read sr-gnn done, num=1797
179700
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=7
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742      0
20480        1    89568  0.983763      0
20968        1    97795  0.983877      0
group done
num=176238, filter_num=3462
read standard sr-gnn results done....
sr-gnn begin...
read sr-gnn done, num=1797
179700
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=7
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742      0
20480        1    89568  0.98376

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



swing item-sim-pair done, pair_num=44361



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




bi-graph recall done, recall_user_num=1752.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [64]:
result

user_id       0       1       2       3      4       5       6      7  \
0           7   79771  113031  115146  117519  57786   64278    4541  10596   
1           8   13933    8563     570   39469   6552   34006    1313  53661   
2           9   67156   19724    7057   11170  59038   21431   77133   2611   
3          29   16401   60886   16751   19520  25034  110573    6378  71421   
4          30  110172   86313   28881   90156  27663   48047   47210   2611   
...       ...     ...     ...     ...     ...    ...     ...     ...    ...   
5362    35395   12638   88109   32500   37383  49944    2436   48961  46110   
5363    35406   52222    8717   19638    9230   9098   56916   18319  96445   
5364    35418   43776   69060   87207   20921  81447    7249   33109  30800   
5365    35429   79490   41178    2213    9408  37872   70253   34551  51467   
5366    35440   62387   59549   88652   19988  82469   56255  111140  76752   

           8  ...     40     41      42     43      44      45      46  \
0     115074  ...  87565  55580  116784  39475  116977   12661   37918   
1        628  ...  53274  24530   28327   7765     204   81055   24397   
2      21504  ...   4230  65790   78822  99225   93192   71750  111008   
3      14038  ...  34009  16300   39785  46260    3881  115735   17952   
4      67634  ...  32925  24732   69717  81417   35804   63007   76405   
...      ...  ...    ...    ...     ...    ...     ...     ...     ...   
5362   32737  ...   8721  26022   88184  43541   78478   36792   21810   
5363   40642  ...   6696  41936   40785  91409    1492   14224   53697   
5364   27500  ...  10234  92765   26789  93262    9341   21799   82273   
5365   79246  ...  91685  57658   71318  92349   56551   39756  107271   
5366   73045  ...     93  64622   21491  95211   27245   23120   71505   

          47      48     49  
0      21622   83649  13845  
1      49155    5507  38435  
2     101172   79951   5750  
3      34213     713   4166  
4      76169  108205  79793  
...      ...     ...    ...  
5362   73514   76480  46300  
5363   48222   43123  10045  
5364   23215   28296  26380  
5365   78869   42600  50313  
5366   27235   91878  57079  

[5367 rows x 51 columns]

# Ranking

The code of this part is not well checked, so you may encounter some problems when running this part

In [51]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM
from deepctr.models.din import DIN
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names, VarLenSparseFeat
from sklearn.metrics import log_loss, roc_auc_score
from tensorflow.python.keras.models import Model, load_model, save_model
from deepctr.layers import custom_objects
import tensorflow as tf
tf.set_random_seed(1234)

## sliding construct training data

In [154]:
def get_history_and_last_click_df(click_df):
    click_df = click_df.sort_values(by=['user_id', 'time'])
    click_last_df = click_df.groupby('user_id').tail(1)

    # 用户只有1个点击时，history为空了，导致训练的时候这个用户不可见, 此时默认一下该用户泄露
    def hist_func(user_df):
        num = len(user_df)
        if num == 1:
            return user_df
        else:
            return user_df[:-1]

    click_history_df = click_df.groupby('user_id').apply(hist_func).reset_index(drop=True)
  
    return click_history_df, click_last_df

def sliding_obtain_training_df(c, is_silding_compute_sim=False):
    all_click, click_q_time =  get_phase_click(c)
           
    # for validation
    compute_mode = 'once' if not is_silding_compute_sim else 'multi'
    
    save_training_path = os.path.join(user_data_dir, 'training', mode, compute_mode, str(c))
    click_history_df = all_click
    recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}
        
    !mkdir -p {save_training_path}
    total_step = 10
    step = 0
    full_sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(click_history_df, recall_methods=recall_methods) 
    pickle.dump(full_sim_pair_dict, open(os.path.join(save_training_path, 'full_sim_pair_dict.pkl'), 'wb'))

   
    step_user_recall_item_dict = {}
    step_strategy_sim_pair_dict = {}

    while step < total_step:
        print('step={}'.format(step))
        click_history_df, click_last_df = get_history_and_last_click_df(click_history_df)  # override click_history_df
        user_item_time_dict = get_user_item_time_dict(click_history_df)

        if is_silding_compute_sim:
            sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(click_history_df, recall_methods=recall_methods) # re-compute
        else:
            sim_pair_dict = full_sim_pair_dict

        user_recall_item_dict = do_multi_recall_results_multi_processing(sim_pair_dict, user_item_time_dict, 
                                                                         ret_type='tuple', recall_methods=recall_methods)

        step_user_recall_item_dict[step] =  user_recall_item_dict
        if  is_silding_compute_sim:
            step_strategy_sim_pair_dict[step] = sim_pair_dict
        step += 1

    pickle.dump(step_user_recall_item_dict, open(os.path.join(save_training_path, 'step_user_recall_item_dict.pkl'), 'wb'))

    if  is_silding_compute_sim:
        pickle.dump(step_strategy_sim_pair_dict, open(os.path.join(save_training_path, 'step_strategy_sim_pair_dict.pkl'), 'wb'))

    # validation/test recall results based on full_sim_pair_dict
    # user-cf depend on sim-user history, so use all-click; test user history will not occur in train, so it's ok
    print('obtain validate/test recall data')
    if mode == 'offline':
        all_user_item_dict = get_user_item_time_dict(all_click) 

        val_user_recall_item_dict = do_multi_recall_results_multi_processing(full_sim_pair_dict, 
                                                                        all_user_item_dict, 
                                                                        target_user_ids=click_test['user_id'].unique(), ret_type='tuple', 
                                                                        recall_methods=recall_methods)
        
        pickle.dump(val_user_recall_item_dict, open(os.path.join(save_training_path, 'val_user_recall_item_dict.pkl'), 'wb'))

In [155]:
# will consume amount of time to finish !!!
for i in range(start_phase, now_phase+1):
    sliding_obtain_training_df(i, is_silding_compute_sim=True)

train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=7
bi-graph item-sim begin
swing item-sim begin
item-cf item-sim begin
user-cf user-sim begin



user-cf user-sim-pair done, pair_num=19801

swing item-sim-pair done, pair_num=45190




bi-graph item-sim-pair done, pair_num=45194

item-cf item-sim-pair done, pair_num=45194
current_len=0
current_len=1
current_len=2
current_len=3
step=0
swing item-sim begin
bi-graph item-sim begin
item-cf item-sim begin
user-cf user-sim begin



user-cf user-sim-pair done, pair_num=19801

swing item-sim-pair done, pair_num=45055

bi-graph item-sim-pair done, pair_num=45183




item-cf item-sim-pair done, pair_num=45183
current_len=0
current_len=1
current_len=2
current_len=3
recall-source-num=4
do recall for user-cf
adjust_type=v2


...


swing recall done, recall_user_num=19883.
do recall for bi-graph
adjust_type=v2


do recall for item-cf
adjust_type=v2


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




user-cf recall done, recall_user_num=19883.
current_len=0
current_len=1
current_len=2
current_len=3
4
aggregate recall results begin....
name=swing, weight=1.0
name=bi-graph, weight=1.0
name=user-cf, weight=1.0
name=item-cf, weight=1.0
step=3
swing item-sim begin
bi-graph item-sim begin
item-cf item-sim begin
user-cf user-sim begin



user-cf user-sim-pair done, pair_num=19883

swing item-sim-pair done, pair_num=42613
obtain validate/test recall data


## ranking data

### organize recall feat

In [18]:
import time
t = (2020, 4, 10, 0, 0, 0, 0, 0, 0)
time_end = time.mktime(t)
max_day, max_hour, max_miniute = 7, 24, 60

def time_info(time_delta):
    import time
    timestamp = time_end * time_delta
    struct_time = time.gmtime(timestamp)
    day, hour, mini = struct_time.tm_wday+1, struct_time.tm_hour+1, struct_time.tm_min+1
    return (day, hour, mini)

def obtain_user_hist_feat(u, user_item_dict):
    user_hist_seq = [i for i, t in user_item_dict[u]]
    user_hist_time_seq = [t for i, t in user_item_dict[u]]
    user_hist_day_seq, user_hist_hour_seq, user_hist_min_seq = zip(*[time_info(t) for i, t in user_item_dict[u]])
    return [user_hist_seq, user_hist_time_seq, list(user_hist_day_seq), list(user_hist_hour_seq), list(user_hist_min_seq)]
  
def organize_recall_feat_each_user(u, recall_items, user_item_dict, strategy_item_sim_dict, phase):
    user_hist_info = obtain_user_hist_feat(u, user_item_dict)
    
    # hist-item similarity with recall items
    hist_num = 3
    recall_items_sum_cf_sim2_hist = []
    recall_items_max_cf_sim2_hist = []
    recall_items_cnt_sim2_hist = []
    
    user_hist_items = user_item_dict[u][::-1][-hist_num:]

    for recall_i, rating in recall_items:
        if rating > 0:
            max_cf_sim2_hist = []
            sum_cf_sim2_hist = []
            cnt_sim2_hist = []
            for hist_i, t in user_hist_items:
                sum_sim_value = 0.0
                max_sim_value = 0.0
               
                for strategy, item_sim_dict in strategy_item_sim_dict.items():
                    strategy_sim_value = item_sim_dict.get(hist_i, {}).get(recall_i, 0.0) + item_sim_dict.get(recall_i, {}).get(hist_i, 0.0)
                    sum_sim_value += strategy_sim_value
                    max_sim_value = max(max_sim_value, strategy_sim_value)
                    
                cnt_sim_value = item_content_sim_dict.get(hist_i, {}).get(recall_i, 0.0) + item_content_sim_dict.get(recall_i, {}).get(hist_i, 0.0)
      
                sum_cf_sim2_hist.append(sum_sim_value)
                max_cf_sim2_hist.append(max_sim_value)
                cnt_sim2_hist.append(cnt_sim_value)

            while len(sum_cf_sim2_hist) < hist_num:
                sum_cf_sim2_hist.append(0.0)
                max_cf_sim2_hist.append(0.0)
                cnt_sim2_hist.append(0.0)
                
        else:
            sum_cf_sim2_hist = [0.0] * hist_num
            max_cf_sim2_hist = [0.0] * hist_num
            cnt_sim2_hist = [0.0] * hist_num

        recall_items_sum_cf_sim2_hist.append(sum_cf_sim2_hist)
        recall_items_max_cf_sim2_hist.append(max_cf_sim2_hist)
        recall_items_cnt_sim2_hist.append(cnt_sim2_hist)
    
    recom_item = []
    for item_rating, sum_cf_sim2_hist, max_cf_sim2_hist, cnt_sim2_hist in zip(recall_items, recall_items_sum_cf_sim2_hist, recall_items_max_cf_sim2_hist, recall_items_cnt_sim2_hist):
        recom_item.append([u, item_rating[0], item_rating[1], phase] + sum_cf_sim2_hist + max_cf_sim2_hist + \
                          cnt_sim2_hist + user_hist_info)
        
    return recom_item

def organize_recall_feat(recall_item_dict, user_item_hist_dict, item_sim_dict, phase):
    recom_columns = ['user_id', 'item_id', 'sim', 'phase'] + \
                      ['sum_sim2int_1', 'sum_sim2int_2', 'sum_sim2int_3'] + \
                             ['max_sim2int_1', 'max_sim2int_2', 'max_sim2int_3']  + \
                        ['cnt_sim2int_1', 'cnt_sim2int_2', 'cnt_sim2int_3'] + \
                          ['hist_item_id', 'hist_time', 'hist_day_id', 'hist_hour_id', 'hist_minute_id']
    recom_item = []
    for u, recall_items in recall_item_dict.items():
        recom_item.extend(organize_recall_feat_each_user(u, recall_items, user_item_hist_dict, item_sim_dict, phase))

    recall_recom_df = pd.DataFrame(recom_item, columns=recom_columns)
    recall_recom_df['sim_rank_score'] = recall_recom_df.groupby('user_id')['sim'].rank(method='first', ascending=True) / topk_num
    
    return recall_recom_df

### organize label 

In [19]:
basic_columns = ['user_id','item_id', 'phase', 'label', ]
time_columns = ['time', 'day_id', 'hour_id', 'minute_id']
hist_columns = ['hist_item_id', 'hist_time', 'hist_day_id', 'hist_hour_id', 'hist_minute_id',]
sim_columns = ['sim', 'sum_sim2int_1', 'sum_sim2int_2', 'sum_sim2int_3'] + \
                             ['max_sim2int_1', 'max_sim2int_2', 'max_sim2int_3', 'sim_rank_score']  + \
                              ['cnt_sim2int_1', 'cnt_sim2int_2', 'cnt_sim2int_3']

use_columns =  basic_columns + hist_columns + sim_columns + time_columns

def organize_label_interact_feat_df(click_last_df, click_last_recall_recom_df, phase, is_consider_cold_start=True):
    dfm_df = pd.merge(click_last_recall_recom_df, click_last_df[['user_id', 'item_id', 'time']], on=['user_id', 'item_id'], how='left') 
    dfm_df['label'] = dfm_df['time'].apply(lambda x: 0.0 if np.isnan(x) else 1.0) # time非空代表该click-item被召回了
    del dfm_df['time']

    # merge_time
    click_last_df['day_id'],  click_last_df['hour_id'], click_last_df['minute_id'] = zip(*click_last_df['time'].apply(time_info))
    dfm_df = pd.merge(dfm_df, click_last_df[['user_id', 'time', 'day_id', 'hour_id', 'minute_id']], on='user_id', how='left')


    # click_last_df里头有些用户的点击没有召回到，即：全部为负样本，导致下采样时，这些用户的负样本可能全被下采样掉了，导致这些用户id丢失；
    # item同理。用户真实点击的item可能没有召回到。
    dfm_df = downsample_by_user(dfm_df)
    dfm_df = dfm_df[use_columns]

    # cold_start_item直接泄露, 这些item可能在infer阶段被recall到，导致item_id缺失
    cold_start_items = set(click_last_df['item_id'].unique()) - set(dfm_df['item_id'].unique())
    if is_consider_cold_start and len(cold_start_items) > 0:
        click_last_cold_start_df = click_last_df[click_last_df['item_id'].isin(cold_start_items)]
        click_last_cold_start_df['label'] = 1.0
        click_last_cold_start_df['phase'] = phase
        for sim_col in sim_columns:
            mean_value = dfm_df[dfm_df['label'] == 1.0][sim_col].mean()
            print('sim_col={}, mean_value={}'.format(sim_col, mean_value))
            click_last_cold_start_df[sim_col] = mean_value
        click_last_cold_start_df = pd.merge(click_last_cold_start_df, dfm_df[['user_id',] + hist_columns], on='user_id', how='left')
    
        print('cold_start_item_num={}, hit_last_cold_start_df_num={}'.format(len(cold_start_items), len(click_last_cold_start_df)))
        dfm_df = dfm_df.append(click_last_cold_start_df[use_columns])

#     dfm_df = sim_process(dfm_df) # TODO, 移动到召回里头呢？
    return dfm_df

def downsample_by_user(df, percent=10):
    '''
    percent:多数类别下采样的数量相对于少数类别样本数量的比例
    '''
    data_pos = df[df['label'] != 0]
    data_neg = df[df['label'] == 0]

    def group_neg_sample_func(group_df):
        total_neg_num = len(group_df)
        sample_num = max(int(total_neg_num * 0.002), 1) # 有些用户召回的数量不足, 取1个
        sample_num = min(sample_num, 5)
        return group_df.sample(n=sample_num, replace=True)

    data_u_neg = data_neg.groupby('user_id', group_keys=False).apply(group_neg_sample_func) # # 保证user全覆盖
    data_i_neg = data_neg.groupby('item_id', group_keys=False).apply(group_neg_sample_func) # 保证item全覆盖
    data_neg = data_u_neg.append(data_i_neg)
    data_neg = data_neg.sort_values(['user_id', 'sim']).drop_duplicates(['user_id', 'item_id'], keep='last')

    data = pd.concat([data_neg, data_pos], ignore_index=True)
    data = data.sample(frac=1.0)
    return data


### organize interact train/val data

1. 先获取steps的recall结果以及对应的 strategy_item_sim_dict

处理训练集：
2. 接着对每个step, 
      进行organize recall feat 
3. 对full_step_df进行organize label操作
 
处理验证集
1.  对验证集进行organize recall feat 
2. 对验证集也进行organize label操作

In [20]:
def organize_train_data_multi_processing(c, is_silding_compute_sim=False, load_from_file=True, total_step=10):
    print('total_step={}'.format(total_step))
    # 1. get recall results
    compute_mode = 'once' if not is_silding_compute_sim else 'multi'
    save_training_path = os.path.join(user_data_dir, 'training', mode, compute_mode, str(c))

    save_result_train_val_path = os.path.join(save_training_path, 'train_val_label_target_id_data.pkl')
    if load_from_file and os.path.exists(save_result_train_val_path):
        return pickle.load(open(save_result_train_val_path, 'rb'))

    all_click, test_q_time = get_phase_click(c)

    click_history_df = all_click

    full_sim_pair_dict = pickle.load(open(os.path.join(save_training_path, 'full_sim_pair_dict.pkl'), 'rb'))
    step_user_recall_item_dict = pickle.load(
        open(os.path.join(save_training_path, 'step_user_recall_item_dict.pkl'), 'rb'))

    if is_silding_compute_sim:
        step_strategy_sim_pair_dict = pickle.load(
            open(os.path.join(save_training_path, 'step_strategy_sim_pair_dict.pkl'), 'rb'))
    print('read recall data done...')

    from multiprocessing import Process, JoinableQueue, Queue

    def convert(click_history_df, click_last_df, user_recall_item_dict, strategy_sim_pair_dict, input_q, result_q):
        step = input_q.get()
        print('step={} begin...'.format(step))
        user_item_time_dict = get_user_item_time_dict(click_history_df)
        # organize recall interact feat
        click_last_recall_recom_df = organize_recall_feat(user_recall_item_dict, user_item_time_dict,
                                                          strategy_sim_pair_dict, c)

        assert len(user_item_time_dict) == len(click_last_recall_recom_df['user_id'].unique()) == len(
            click_last_df['user_id'].unique())

        train_full_df = organize_label_interact_feat_df(click_last_df, click_last_recall_recom_df, c)
        train_full_df['step'] = step
        print(train_full_df['label'].value_counts())
        result_q.put(train_full_df)
        input_q.task_done()
        assert 'sim' in train_full_df.columns

    input_q = JoinableQueue()
    result_q = Queue()

    processes = []
    for step in range(total_step):
        input_q.put(step)
        click_history_df, click_last_df = get_history_and_last_click_df(click_history_df)  # override click_history_df
        user_recall_item_dict = step_user_recall_item_dict[step]
        strategy_sim_pair_dict = step_strategy_sim_pair_dict[step] if is_silding_compute_sim else full_sim_pair_dict

        processes.append(Process(target=convert, args=(click_history_df, click_last_df,
                                                       user_recall_item_dict, strategy_sim_pair_dict,
                                                       input_q, result_q)))
        processes[-1].daemon = True
        processes[-1].start()

    input_q.join()

    train_full_df_list = []
    while len(train_full_df_list) != total_step:
        train_full_df = result_q.get()
        train_full_df_list.append(train_full_df)

    for p in processes:
        p.terminate()
        p.join()

    print('obtain train data done....')

    assert len(train_full_df_list) == total_step

    if mode == 'offline':
        train_full_df = pd.concat(train_full_df_list, ignore_index=True)
        # valid data
        print('begin obtain validate data...')
        val_user_item_dict = get_user_item_time_dict(click_test)  # click_test as history
        val_user_recall_item_dict = pickle.load(
            open(os.path.join(save_training_path, 'val_user_recall_item_dict.pkl'), 'rb'))

        phase_val_last_click_answer_df = pd.read_csv('{}/{}-{}.csv'.format(offline_answer_path, infer_answer_file_prefix, c),
                                                     header=None, names=['user_id', 'item_id', 'time'])
        # organize recall interact feat
        phase_val_last_click_recall_recom_df = organize_recall_feat(val_user_recall_item_dict, val_user_item_dict,
                                                                    full_sim_pair_dict, c)

        val_full_df = organize_label_interact_feat_df(phase_val_last_click_answer_df,
                                                      phase_val_last_click_recall_recom_df, c, False)
        val_target_uids = phase_val_last_click_answer_df['user_id'].unique()

        save_train_val_path = os.path.join(save_training_path, 'train_val_label_target_id_data.pkl')
        pickle.dump([train_full_df, val_full_df, val_target_uids], open(save_train_val_path, 'wb'))

        return train_full_df, val_full_df, val_target_uids

    else:
        print('online')
        train_full_df = pd.concat(train_full_df_list, ignore_index=True)
        save_train_val_path = os.path.join(save_training_path, 'train_val_label_target_id_data.pkl')
        pickle.dump(train_full_df, open(save_train_val_path, 'wb'))
        return train_full_df



### word2vec feat

In [21]:
from gensim.models.word2vec import *
w2v_dim = 32
def get_word2vec_feat(full_user_item_df):
    import time
    seq_list = full_user_item_df['hist_item_id'].apply(lambda x:[str(i) for i in x]).values
    print(seq_list.shape)
    begin_time = time.time()
    model = Word2Vec(seq_list, size=w2v_dim, window=5, min_count=0, workers=40, sg=0, hs=1)
    end_time = time.time()
    run_time = end_time-begin_time

    print ('该循环程序运行时间：',round(run_time,2)) #该循环程序运行时间： 1.4201874732

    word2idx = {"_PAD": 0} # 初始化 `[word : token]` 字典，后期 tokenize 语料库就是用该词典。
    vocab_list = [(k, model.wv[k]) for k, v in model.wv.vocab.items()]
    word2vec_item_embed_dict = dict(vocab_list)
    
    return word2vec_item_embed_dict

### fill item feat

In [22]:
def fill_item_feat():
    all_click_feat_df = pd.merge(online_total_click, processed_item_feat_df, on='item_id', how='left')
    # 缺失值
    missed_items = all_click_feat_df[all_click_feat_df['txt_embed_0'].isnull()]['item_id'].unique()
    user_item_time_hist_dict = get_user_item_time_dict(online_total_click)
    
    # co-occurance
    co_occur_dict = {}
    window = 5
    def cal_occ(sentence):
        for i,word in enumerate(sentence):
            hist_len = len(sentence)
            co_occur_dict.setdefault(word, {})
            for j in range(max(i-window,0), min(i+window, hist_len)):
                if j == i or word == sentence[j]: continue
                loc_weight = (0.9**abs(i-j)) 
                co_occur_dict[word].setdefault(sentence[j], 0)
                co_occur_dict[word][sentence[j]] += loc_weight

    for u, hist_item_times in user_item_time_hist_dict.items():
        hist_items = [i for i, t in hist_item_times]
        cal_occ(hist_items)
    
    # fill
    miss_item_content_vec_dict = {}
    for miss_item in missed_items:
        co_occur_item_dict = co_occur_dict[miss_item]
        weighted_vec = np.zeros(256)
        sum_weight = 0.0
        for co_item, weight in co_occur_item_dict.items():

            if co_item in item_content_vec_dict:
                sum_weight += weight
                co_item_vec = item_content_vec_dict[co_item]
                weighted_vec += weight*co_item_vec

        weighted_vec /= sum_weight
        txt_item_feat_np = weighted_vec[0:128] / np.linalg.norm(weighted_vec[0:128])
        img_item_feat_np = weighted_vec[128:] / np.linalg.norm(weighted_vec[128:])
        cnt_vec = np.concatenate([txt_item_feat_np,  img_item_feat_np])
        miss_item_content_vec_dict[miss_item] = cnt_vec
    
    miss_item_feat_df = pd.DataFrame()
    miss_item_feat_df[item_dense_feat] = pd.DataFrame(miss_item_content_vec_dict.values(), 
                                                      columns=item_dense_feat)
    miss_item_feat_df['item_id'] = list(miss_item_content_vec_dict.keys())
    miss_item_feat_df = miss_item_feat_df[['item_id'] + item_dense_feat]
    
    return miss_item_feat_df, miss_item_content_vec_dict

### organize raw user-item feat 

In [23]:
def process_item_feat(item_feat_df):
    processed_item_feat_df = item_feat_df.copy()
    txt_dense_feat = ['txt_embed_'+str(i) for i in range(128)] 
    img_dense_feat = ['img_embed_'+str(i) for i in range(128)]
    dense_feat = txt_dense_feat + img_dense_feat
    # norm
    txt_item_feat_np = processed_item_feat_df[txt_dense_feat].values
    img_item_feat_np = processed_item_feat_df[img_dense_feat].values
    txt_item_feat_np = txt_item_feat_np / np.linalg.norm(txt_item_feat_np, axis=1, keepdims=True)
    img_item_feat_np = img_item_feat_np / np.linalg.norm(img_item_feat_np, axis=1, keepdims=True)
    processed_item_feat_df[txt_dense_feat] = pd.DataFrame(txt_item_feat_np, columns=txt_dense_feat)
    processed_item_feat_df[img_dense_feat] = pd.DataFrame(img_item_feat_np, columns=img_dense_feat)

    # item_feat_dict = dict(zip(processed_item_feat_df['item_id'], processed_item_feat_df[dense_feat].values))
    return processed_item_feat_df, dense_feat

def process_user_feat(user_feat_df):
    # sparse encoder
    user_sparse_feat = ['age_level','gender','city_level']
    return user_feat_df, user_sparse_feat

In [24]:
def sparse_feat_fit(total_click):
    global feat_lbe_dict, item_raw_id2_idx_dict, user_raw_id2_idx_dict
    
    from sklearn.preprocessing import LabelEncoder, MinMaxScaler
    # sparse features one-hot
    feat_lbe_dict = {}
    for feat in sparse_feat:
        if feat in time_feat: continue
        lbe = LabelEncoder()
        lbe.fit(total_click[feat].astype(str))
        feat_lbe_dict[feat] = lbe
    
    item_raw_id2_idx_dict = dict(zip(feat_lbe_dict['item_id'].classes_, 
                     feat_lbe_dict['item_id'].transform(feat_lbe_dict['item_id'].classes_)+1, )) # 得到字典
    user_raw_id2_idx_dict = dict(zip(feat_lbe_dict['user_id'].classes_, 
                     feat_lbe_dict['user_id'].transform(feat_lbe_dict['user_id'].classes_)+1, )) # 得到字典
    

def sparse_feat_transform(df):
    df['hist_item_id'] = df['hist_item_id'].apply(lambda seq: [item_raw_id2_idx_dict[str(x)] for x in seq])
    df['seq_length'] = df['hist_item_id'].apply(lambda hist: min(max_seq_len, len(hist)))
    df['seq_weight'] = df['hist_item_id'].apply(lambda hist: [0.9**(len(hist)-loc) for loc, item in enumerate(hist)])

    for hist_id in var_len_feat: 
        df[hist_id] = tf.keras.preprocessing.sequence.pad_sequences(df[hist_id], 
                                                  value=0, maxlen=max_seq_len, truncating='pre', padding='post').tolist()
        
    df['seq_weight'] = tf.keras.preprocessing.sequence.pad_sequences(df['seq_weight'], 
                                                  value=0, maxlen=max_seq_len, truncating='pre', padding='post', dtype=np.float32).tolist()
    df['seq_weight'] = df['seq_weight'].apply(lambda weights: [[w] for w in weights])
    
    for feat in sparse_feat:
        print(feat)
        if feat in time_feat: continue
        df[feat] = feat_lbe_dict[feat].transform(df[feat].astype(str))+1
    return df


In [48]:
def fillna(df, sparse_feat, dense_feat):
  for sp in sparse_feat:
    df[sp].fillna('-1', inplace=True)
    
  for ds in dense_feat:
    df[ds].fillna(0.0, inplace=True) # all_click_user_item_df[ds].mean()
  return df
  
def organize_user_item_feat(df, item_feat_df, sparse_feat, dense_feat, 
                                        is_interest=True, is_w2v=False):
    
    full_user_item_df = pd.merge(df, item_feat_df, how='left', on='item_id')
    full_user_item_df = fillna(full_user_item_df, sparse_feat, dense_feat)
    print('origin data done')
  
    if is_interest:
        # history interest
        full_user_item_df = obtain_user_hist_interest_feat(full_user_item_df, item_content_vec_dict)
        print('interest done')
    
    if is_w2v:
        organize_word2vec_feat(full_user_item_df, word2vec_item_embed_dict, word2vec_user_embed_dict)
        print('word2vec done')


    full_user_item_df = sparse_feat_transform(full_user_item_df)
    
    return full_user_item_df


def obtain_user_hist_interest_feat(full_user_item_df, item_vec_dict):
    def weighted_agg_content(hist_item_id_list):

        weighted_content = np.zeros(128*2)
        hist_num = len(hist_item_id_list)
        for loc, i in enumerate(hist_item_id_list):
            loc_weight = (0.9**(hist_num-loc)) 
            if i in item_vec_dict:
                weighted_content += loc_weight*item_vec_dict[i]
        return weighted_content

    user_interest_vec = full_user_item_df['hist_item_id'].apply(weighted_agg_content).tolist()
    user_interest_df = pd.DataFrame(user_interest_vec, columns=['interest_'+col for col in item_dense_feat])
    
    full_user_item_df[user_interest_df.columns] = user_interest_df

    # begin compute degree
    target_item_vec = full_user_item_df[item_dense_feat].values
    user_interest_vec = np.array(user_interest_vec) 
    
    txt_interest_degree_array = target_item_vec[:, 0:128] * user_interest_vec[:, 0:128]
    txt_interest_degree_list = np.sum(txt_interest_degree_array, axis=1)
    full_user_item_df['txt_interest_degree'] = txt_interest_degree_list.tolist()
    
    img_interest_degree_array = target_item_vec[:, 128:] * user_interest_vec[:, 128:]
    img_interest_degree_list = np.sum(img_interest_degree_array, axis=1)
    full_user_item_df['img_interest_degree'] = img_interest_degree_list.tolist()
    
    full_user_item_df['interest_degree'] =  full_user_item_df['img_interest_degree']  + full_user_item_df['img_interest_degree'] 
    
    for f in ['interest_'+col for col in item_dense_feat]+['img_interest_degree', 'img_interest_degree', 'interest_degree']:
        full_user_item_df[f].fillna(0.0, inplace=True)
    print('obtain user dynamic feat done')
    
    def hist_2_target_cnt(hist_target_item_list, hist_no):
        target_item = hist_target_item_list[-1]
        if target_item not in item_content_vec_dict:
            return [0.0, 0.0, 0.0]

        hist_target_item_list = hist_target_item_list[: -1]

        if len(hist_target_item_list) >= hist_no:
            hist_item = hist_target_item_list[-hist_no]
            if hist_item in item_content_vec_dict:
                txt_cnt_sim = np.dot(item_content_vec_dict[target_item][0:128], item_content_vec_dict[hist_item][0:128])
                img_cnt_sim = np.dot(item_content_vec_dict[target_item][128:], item_content_vec_dict[hist_item][128:])
                return txt_cnt_sim, img_cnt_sim, txt_cnt_sim + img_cnt_sim

        return [0.0, 0.0, 0.0]

    hist_target_items_series = full_user_item_df['hist_item_id'] + full_user_item_df['item_id'].apply(lambda x:[x])
    full_user_item_df['txt_cnt_sim_last_1'], full_user_item_df['img_cnt_sim_last_1'], full_user_item_df['cnt_sim_last_1']  = zip(*hist_target_items_series.apply(lambda x: hist_2_target_cnt(x, 1)))
    full_user_item_df['txt_cnt_sim_last_2'], full_user_item_df['img_cnt_sim_last_2'], full_user_item_df['cnt_sim_last_2']  = zip(*hist_target_items_series.apply(lambda x: hist_2_target_cnt(x, 2)))
    full_user_item_df['txt_cnt_sim_last_3'], full_user_item_df['img_cnt_sim_last_3'], full_user_item_df['cnt_sim_last_3'] = zip(*hist_target_items_series.apply(lambda x: hist_2_target_cnt(x, 3)))
    
    
    def hist_2_target_time_diff(hist_time_list, hist_num=3):
        target_time = hist_time_list[-1]
        hist_time_list = hist_time_list[: -1]
        
        hist_time_diff = []
        for hist_time in hist_time_list[::-1][0:hist_num]:
            diff_time = target_time - hist_time
            hist_time_diff.append(diff_time)
            
        while len(hist_time_diff) != hist_num:
            hist_time_diff.append(0.1)

        return hist_time_diff
    hist_target_time_series = full_user_item_df['hist_time'] + full_user_item_df['time'].apply(lambda x:[x])
    full_user_item_df['time_diff_1'], full_user_item_df['time_diff_2'], full_user_item_df['time_diff_3'] = zip(*hist_target_time_series.apply(hist_2_target_time_diff))
    
    return full_user_item_df


def organize_word2vec_feat(full_user_item_df, w2v_item_embed_dict, w2v_user_embed_dict):
    
    def lookup_item_word2vec_embed(item_id):
            return w2v_item_embed_dict.get(str(item_id), np.zeros(w2v_dim)).tolist()
        
    def lookup_user_word2vec_embed(user_id):
            return w2v_user_embed_dict.get(str(user_id), np.zeros(w2v_dim)).tolist()
    
    def hist_2_target_w2v(hist_target_item_list, hist_no):
        target_item = hist_target_item_list[-1]
        if str(target_item) not in w2v_item_embed_dict:
            return 0.0

        hist_target_item_list = hist_target_item_list[: -1]

        if len(hist_target_item_list) >= hist_no:
            hist_item = hist_target_item_list[-hist_no]
            if str(hist_item) in w2v_item_embed_dict:
                w2v_sim = np.dot(w2v_item_embed_dict[str(target_item)], w2v_item_embed_dict[str(hist_item)])
                return w2v_sim
        return 0.0
    
    hist_target_items_series = full_user_item_df['hist_item_id'] + full_user_item_df['item_id'].apply(lambda x:[x])
    full_user_item_df['w2v_sim_last_1'] = hist_target_items_series.apply(lambda x: hist_2_target_w2v(x, 1))
    full_user_item_df['w2v_sim_last_2'] = hist_target_items_series.apply(lambda x: hist_2_target_w2v(x, 2))
    full_user_item_df['w2v_sim_last_3'] = hist_target_items_series.apply(lambda x: hist_2_target_w2v(x, 3))
    
    item_w2v_embed_list = full_user_item_df['item_id'].apply(lookup_item_word2vec_embed).tolist() # target_item_id
    user_w2v_embed_list = full_user_item_df['user_id'].apply(lookup_user_word2vec_embed).tolist() # target_user_id
    w2v_sim = np.sum(np.array(user_w2v_embed_list) * np.array(item_w2v_embed_list), axis=1)
    
    
    item_w2v_cols= ['item_w2v_embed_{}'.format(i) for i in range(w2v_dim)]
    item_w2v_pd = pd.DataFrame(item_w2v_embed_list, columns=item_w2v_cols)
    
    user_w2v_cols= ['user_w2v_embed_{}'.format(i) for i in range(w2v_dim)]
    user_w2v_pd = pd.DataFrame(user_w2v_embed_list, columns=user_w2v_cols)
    
    
    full_user_item_df[item_w2v_cols] = item_w2v_pd
    full_user_item_df[user_w2v_cols] = user_w2v_pd
    full_user_item_df['w2v_sim'] = w2v_sim
    
    return full_user_item_df

### running to prepare data

In [164]:
target_phase = 8

In [165]:
if mode == 'offline':
    train_full_df_dict = {}
    val_full_df_dict = {}
    for i in [target_phase]:
        train_full_df, val_full_df, val_target_uids = organize_train_data_multi_processing(i, is_silding_compute_sim=True, 
                                                                                           load_from_file=True)
        train_full_df_dict[i] = train_full_df
        val_full_df_dict[i] = val_full_df

In [28]:
if mode == 'online':
    online_train_full_df_dict = {}
    for i in range(start_phase, now_phase+1):
        print('phase={} start'.format(i))
        if i in online_train_full_df_dict: continue
        online_train_full_df = organize_train_data_multi_processing(i, is_silding_compute_sim=True, load_from_file=True)
        online_train_full_df_dict[i] = online_train_full_df

phase=7 start
total_step=10
phase=8 start
total_step=10
phase=9 start
total_step=10


In [30]:
processed_item_feat_df, item_dense_feat = process_item_feat(item_feat_df)
item_content_vec_dict = dict(zip(processed_item_feat_df['item_id'], processed_item_feat_df[item_dense_feat].values))

In [40]:
is_fill_missing = True
if is_fill_missing:
    miss_item_feat_df, miss_item_content_vec_dict = fill_item_feat()
    processed_item_feat_df = processed_item_feat_df.append(miss_item_feat_df)
    processed_item_feat_df = processed_item_feat_df.reset_index(drop=True)
    item_content_vec_dict.update(miss_item_content_vec_dict)

In [41]:
max_seq_len = 10
time_feat = ['day_id', 'hour_id'] #, 'minute_id']  # no need to sparse encoder
time_vocab_map = {'day_id': max_day, 'minute_id': max_miniute, 'hour_id': max_hour}

sparse_feat = ['user_id', 'item_id',] + time_feat # + user_sparse_feat
user_interest_dense_feat = ['interest_'+col for col in item_dense_feat] + ['interest_degree', 'txt_interest_degree', 'img_interest_degree',]
# sim_dense_feat =  ['sim', 'exp_sim', 'sim2int_1', 'sim2int_2', 'sim2int_3'] + ['cnt_sim2int_1', 'cnt_sim2int_2', 'cnt_sim2int_3'] # , 'sim_rank_score']
sim_dense_feat = ['sim', 'sum_sim2int_1', 'sum_sim2int_2', 'sum_sim2int_3'] + \
                             ['max_sim2int_1', 'max_sim2int_2', 'max_sim2int_3', 'sim_rank_score']  + \
                              ['cnt_sim2int_1', 'cnt_sim2int_2', 'cnt_sim2int_3']

hist_cnt_sim_feat = ['txt_cnt_sim_last_1', 'img_cnt_sim_last_1', 'cnt_sim_last_1'] + \
                            ['txt_cnt_sim_last_2', 'img_cnt_sim_last_2', 'cnt_sim_last_2'] + \
                            ['txt_cnt_sim_last_3', 'img_cnt_sim_last_3', 'cnt_sim_last_3'] 

hist_time_diff_feat = ['time_diff_1', 'time_diff_2', 'time_diff_3']

w2v_sim_feat = ['w2v_sim_last_1', 'w2v_sim_last_2', 'w2v_sim_last_3']

user_w2v_embed_feat = ['user_w2v_embed_{}'.format(i) for i in range(128)]
item_w2v_embed_feat = ['item_w2v_embed_{}'.format(i) for i in range(128)]
w2v_user_item_feat = ['w2v_sim'] + user_w2v_embed_feat + item_w2v_embed_feat
   

item_degree_feat = ['item_degree', 'item_vs_degree_mean', 'item_vs_25_degree', 'item_vs_50_degree', 'item_vs_75_degree'] 
item_time_feat =  ['item_id_first_time', 'item_id_last_time', 'item_id_day_count', 'item_id_hour_count']
item_statistic_feat = item_degree_feat + item_time_feat

user_degree_feat = ['user_degree', 'user_vs_degree_mean', 'user_vs_25_degree', 'user_vs_50_degree', 'user_vs_75_degree'] 
user_time_feat =  ['user_id_first_time', 'user_id_last_time', 'user_id_day_count', 'user_id_hour_count']
user_statistic_feat = user_degree_feat + user_time_feat
                               
dense_feat = item_dense_feat  +  sim_dense_feat # + item_statistic_feat
var_len_feat = ['hist_item_id'] +  ['hist_{}'.format(feat) for feat in time_feat]

In [42]:
sparse_feat_fit(online_total_click)

In [43]:
if mode == 'online':
    train_full_df = online_train_full_df_dict[target_phase]
    if isinstance(train_full_df, list):
        train_full_df = train_full_df[0]
else:
    train_full_df = train_full_df_dict[target_phase]
    val_full_df = val_full_df_dict[target_phase]

In [44]:
word2vec_item_embed_dict = get_word2vec_feat(train_full_df)

(689040,)
该循环程序运行时间： 96.33


In [52]:
train_final_df = organize_user_item_feat(train_full_df, processed_item_feat_df, 
                                         sparse_feat, dense_feat, is_w2v=True, is_interest=True)

origin data done
obtain user dynamic feat done
interest done
word2vec done
user_id
item_id
day_id
hour_id


In [66]:
val_final_df = organize_user_item_feat(val_full_df, processed_item_feat_df, 
                                       sparse_feat, dense_feat, is_w2v=True, is_interest=True)

origin data done
obtain user dynamic feat done
interest done
word2vec done
user_id
item_id
day_id
hour_id


## ranking model

### lightgbm

In [53]:
import lightgbm as lgb
import matplotlib.pyplot as plt
lgb_cols = dense_feat  + user_interest_dense_feat + hist_cnt_sim_feat + hist_time_diff_feat + w2v_sim_feat # + use_kmeans_feats  # ['user_degree'] # ['item_count',]  #, 'first_time', 'last_time'] # item_statistic_feat + time_feat 

In [68]:
# auc: 0.896453
clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
        max_depth=-1, n_estimators=300, 
        subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
        learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs=-1) # 300epoch, best, 0.882898, dense_feat  + hist_cnt_sim_feat user_interest_dense_feat 

if mode == 'offline':
    clf.fit(train_final_df[lgb_cols],  train_final_df['label'], 
           eval_set=[(val_final_df[lgb_cols], val_final_df['label'])],
           eval_metric='auc',   early_stopping_rounds=50, ) 
else:
    clf.fit(train_final_df[lgb_cols],  train_final_df['label']) 

[1]	valid_0's auc: 0.789153	valid_0's binary_logloss: 0.102173
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.806444	valid_0's binary_logloss: 0.101566
[3]	valid_0's auc: 0.814327	valid_0's binary_logloss: 0.100949
[4]	valid_0's auc: 0.816709	valid_0's binary_logloss: 0.10037
[5]	valid_0's auc: 0.821966	valid_0's binary_logloss: 0.0998648
[6]	valid_0's auc: 0.823212	valid_0's binary_logloss: 0.0992914
[7]	valid_0's auc: 0.826434	valid_0's binary_logloss: 0.0988067
[8]	valid_0's auc: 0.827479	valid_0's binary_logloss: 0.0982653
[9]	valid_0's auc: 0.828117	valid_0's binary_logloss: 0.0976873
[10]	valid_0's auc: 0.827339	valid_0's binary_logloss: 0.0971543
[11]	valid_0's auc: 0.829847	valid_0's binary_logloss: 0.0966899
[12]	valid_0's auc: 0.829267	valid_0's binary_logloss: 0.0962361
[13]	valid_0's auc: 0.828941	valid_0's binary_logloss: 0.0957797
[14]	valid_0's auc: 0.830756	valid_0's binary_logloss: 0.0953821
[15]	valid_0's auc: 0.830027	valid_0's bina

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=20, min_child_weight=50, min_split_gain=0.0,
               n_estimators=300, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=2018, reg_alpha=0.0, reg_lambda=1, silent=True,
               subsample=0.7, subsample_for_bin=200000, subsample_freq=1)

In [101]:
train_final_df.sort_values(by=['user_id'], inplace=True)
g_train =  train_final_df.groupby(['user_id'], as_index=False).count()["label"].values

if mode == 'offline':
    val_final_df.sort_values(by=['user_id'], inplace=True)
    g_val = val_final_df.groupby(['user_id'], as_index=False).count()["label"].values


lgb_rank = lgb.LGBMRanker(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
        max_depth=-1, n_estimators=300, 
        subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
        learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs=-1) # 300epoch, best, 0.882898, dense_feat  + hist_cnt_sim_feat user_interest_dense_feat

if mode == 'offline':
    lgb_rank.fit(train_final_df[lgb_cols],  train_final_df['label'], group=g_train,
           eval_set=[(val_final_df[lgb_cols], val_final_df['label'])], eval_group=[g_val], 
           eval_at=[50], eval_metric=['auc',],  
           early_stopping_rounds=50, ) 
else:
    lgb_rank.fit(train_final_df[lgb_cols],  train_final_df['label'], group=g_train,)

[1]	valid_0's auc: 0.786356	valid_0's binary_logloss: 0.10217
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.804719	valid_0's binary_logloss: 0.101562
[3]	valid_0's auc: 0.811087	valid_0's binary_logloss: 0.100949
[4]	valid_0's auc: 0.8139	valid_0's binary_logloss: 0.100373
[5]	valid_0's auc: 0.815925	valid_0's binary_logloss: 0.0998699
[6]	valid_0's auc: 0.819037	valid_0's binary_logloss: 0.0992986
[7]	valid_0's auc: 0.824667	valid_0's binary_logloss: 0.0988134
[8]	valid_0's auc: 0.824252	valid_0's binary_logloss: 0.0982705
[9]	valid_0's auc: 0.826179	valid_0's binary_logloss: 0.0976908
[10]	valid_0's auc: 0.825209	valid_0's binary_logloss: 0.0971649
[11]	valid_0's auc: 0.828151	valid_0's binary_logloss: 0.0967054
[12]	valid_0's auc: 0.827276	valid_0's binary_logloss: 0.0962453
[13]	valid_0's auc: 0.828525	valid_0's binary_logloss: 0.0957923
[14]	valid_0's auc: 0.832094	valid_0's binary_logloss: 0.0953885
[15]	valid_0's auc: 0.831418	valid_0's binary

LGBMRanker(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
           importance_type='split', learning_rate=0.01, max_depth=-1,
           min_child_samples=20, min_child_weight=50, min_split_gain=0.0,
           n_estimators=300, n_jobs=-1, num_leaves=31, objective='binary',
           random_state=2018, reg_alpha=0.0, reg_lambda=1, silent=True,
           subsample=0.7, subsample_for_bin=200000, subsample_freq=1)

In [110]:
feat_importance = pd.Series(clf.feature_importances_, index=lgb_cols).sort_values(ascending=False).reset_index().rename(columns={'index':'feat', 0:'importance'})
feat_importance

feat  importance
0    item_id_same_cluster_rank_percent         502
1                                  sim         422
2    user_id_same_cluster_rank_percent         373
3                          time_diff_1         361
4                       sim_rank_score         251
..                                 ...         ...
538              interest_txt_embed_69           0
539              interest_txt_embed_70           0
540              interest_txt_embed_71           0
541              interest_txt_embed_72           0
542                        txt_embed_0           0

[543 rows x 2 columns]

### DIN

References: DeepCTR, Easy-to-use,Modular and Extendible package of deep-learning based CTR models, https://github.com/shenweichen/DeepCTR

In [55]:
item_cnt = len(item_raw_id2_idx_dict)
item_embed_np = np.zeros((item_cnt+1, 256))
for raw_id, idx in item_raw_id2_idx_dict.items():
    vec = item_content_vec_dict[int(raw_id)]
    item_embed_np[idx, :] = vec
# np.save(open(sr_gnn_dir + '/data/item_embed_mat.npy', 'wb'), item_embed_np)

In [56]:
def get_init_user_embed(target_phase, is_use_whole_click=True):
    global user_embed_np
    all_click, click_q_time = get_phase_click(target_phase)
    if is_use_whole_click:
        phase_click = get_whole_phase_click(all_click, click_q_time)
    else:
        phase_click = all_click

    user_item_time_hist_dict = get_user_item_time_dict(phase_click)

    def weighted_agg_content(hist_item_id_list):
        weighted_vec = np.zeros(128*2)
        hist_num = len(hist_item_id_list)
        sum_weight = 0.0
        for loc, (i,t) in enumerate(hist_item_id_list):
            loc_weight = (0.9**(hist_num-loc)) 
            if i in item_content_vec_dict:
                sum_weight += loc_weight
                weighted_vec += loc_weight*item_content_vec_dict[i]
        if sum_weight != 0:
            weighted_vec /= sum_weight
            txt_item_feat_np = weighted_vec[0:128] / np.linalg.norm(weighted_vec[0:128])
            img_item_feat_np = weighted_vec[128:] / np.linalg.norm(weighted_vec[128:])
            weighted_vec = np.concatenate([txt_item_feat_np,  img_item_feat_np])
        else:
            print('zero weight...')
        return weighted_vec
    user_cnt = len(user_raw_id2_idx_dict)
    user_embed_np = np.zeros((user_cnt+1, 256))
    for raw_id, idx in user_raw_id2_idx_dict.items():
        if int(raw_id) in user_item_time_hist_dict:
            hist = user_item_time_hist_dict[int(raw_id)]
            vec = weighted_agg_content(hist)
            user_embed_np[idx, :] = vec
    # np.save(open(sr_gnn_dir + '/data/user_embed_mat.npy', 'wb'), user_embed_np)

In [57]:
from tensorflow.python.keras.initializers import RandomNormal, Constant
from deepctr.inputs import  build_input_features,create_embedding_matrix,SparseFeat,VarLenSparseFeat,DenseFeat,embedding_lookup,get_dense_input,varlen_embedding_lookup,get_varlen_pooling_list,combined_dnn_input
from tensorflow.python.keras.layers import Embedding, Input, Flatten
from tensorflow.python.keras.regularizers import l2

def kdd_create_embedding_matrix(feature_columns, l2_reg, init_std, seed, prefix="", seq_mask_zero=True):
    sparse_feature_columns = list(
        filter(lambda x: isinstance(x, SparseFeat), feature_columns)) if feature_columns else []
    varlen_sparse_feature_columns = list(
        filter(lambda x: isinstance(x, VarLenSparseFeat), feature_columns)) if feature_columns else []
    sparse_emb_dict = kdd_create_embedding_dict(sparse_feature_columns, varlen_sparse_feature_columns, init_std, seed,
                                            l2_reg, prefix=prefix + 'sparse', seq_mask_zero=seq_mask_zero)
    return sparse_emb_dict


def kdd_create_embedding_dict(sparse_feature_columns, varlen_sparse_feature_columns, init_std, seed, l2_reg,
                          prefix='sparse_', seq_mask_zero=True):
    sparse_embedding = {}
    for feat in sparse_feature_columns:
        embed_initializer = RandomNormal(mean=0.0, stddev=init_std, seed=seed)
        if feat.embedding_name == 'user_id':
            print('init user embed')
            embed_initializer = Constant(user_embed_np)
        if feat.embedding_name == 'item_id':
            print('init item embed')
            embed_initializer = Constant(item_embed_np)
        sparse_embedding[feat.embedding_name] = Embedding(feat.vocabulary_size, feat.embedding_dim,
                                                                       embeddings_initializer=embed_initializer,
#                                                                        embeddings_regularizer=l2(l2_reg),
                                                                       name=prefix + '_emb_' + feat.embedding_name)

    if varlen_sparse_feature_columns and len(varlen_sparse_feature_columns) > 0:
        for feat in varlen_sparse_feature_columns:
            embed_initializer = RandomNormal(mean=0.0, stddev=init_std, seed=seed)
            if feat.embedding_name == 'user_id':
                print('init user embed')
                embed_initializer = Constant(user_embed_np)
            if feat.embedding_name == 'item_id':
                print('init item embed')
                embed_initializer = Constant(item_embed_np)
            sparse_embedding[feat.embedding_name] = Embedding(feat.vocabulary_size, feat.embedding_dim,
                                                              embeddings_initializer=embed_initializer,
#                                                               embeddings_regularizer=l2(l2_reg),
                                                              name=prefix + '_seq_emb_' + feat.name,
                                                              mask_zero=seq_mask_zero)
    return sparse_embedding

In [58]:
# -*- coding:utf-8 -*-
from tensorflow.python.keras.layers import Dense,Concatenate, Flatten
from tensorflow.python.keras.models import Model

from deepctr.inputs import  build_input_features,create_embedding_matrix,SparseFeat,VarLenSparseFeat,DenseFeat,embedding_lookup,get_dense_input,varlen_embedding_lookup,get_varlen_pooling_list,combined_dnn_input
from deepctr.layers.core import DNN, PredictionLayer
from deepctr.layers.sequence import AttentionSequencePoolingLayer
from deepctr.layers.utils import concat_func, NoMask


def KDD_DIN(dnn_feature_columns, history_feature_list, dnn_use_bn=False,
        dnn_hidden_units=(200, 80), dnn_activation='relu', att_hidden_size=(80, 40), att_activation="dice",
        att_weight_normalization=False, l2_reg_dnn=0, l2_reg_embedding=1e-6, dnn_dropout=0, init_std=0.0001, seed=1024,
        task='binary'):
    """Instantiates the Deep Interest Network architecture.

    :param dnn_feature_columns: An iterable containing all the features used by deep part of the model.
    :param history_feature_list: list,to indicate  sequence sparse field
    :param dnn_use_bn: bool. Whether use BatchNormalization before activation or not in deep net
    :param dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of deep net
    :param dnn_activation: Activation function to use in deep net
    :param att_hidden_size: list,list of positive integer , the layer number and units in each layer of attention net
    :param att_activation: Activation function to use in attention net
    :param att_weight_normalization: bool.Whether normalize the attention score of local activation unit.
    :param l2_reg_dnn: float. L2 regularizer strength applied to DNN
    :param l2_reg_embedding: float. L2 regularizer strength applied to embedding vector
    :param dnn_dropout: float in [0,1), the probability we will drop out a given DNN coordinate.
    :param init_std: float,to use as the initialize std of embedding vector
    :param seed: integer ,to use as random seed.
    :param task: str, ``"binary"`` for  binary logloss or  ``"regression"`` for regression loss
    :return: A Keras model instance.

    """


    features = build_input_features(dnn_feature_columns)

    sparse_feature_columns = list(filter(lambda x:isinstance(x,SparseFeat),dnn_feature_columns)) if dnn_feature_columns else []
    dense_feature_columns = list(
        filter(lambda x: isinstance(x, DenseFeat), dnn_feature_columns)) if dnn_feature_columns else []
    varlen_sparse_feature_columns = list(filter(lambda x: isinstance(x, VarLenSparseFeat), dnn_feature_columns)) if dnn_feature_columns else []


    history_feature_columns = []
    sparse_varlen_feature_columns = []
    history_fc_names = list(map(lambda x: "hist_" + x, history_feature_list))
    for fc in varlen_sparse_feature_columns:
        feature_name = fc.name
        if feature_name in history_fc_names:
            history_feature_columns.append(fc)
        else:
            sparse_varlen_feature_columns.append(fc)


    inputs_list = list(features.values())


    embedding_dict = kdd_create_embedding_matrix(dnn_feature_columns, l2_reg_embedding, init_std, seed, prefix="")


    query_emb_list = embedding_lookup(embedding_dict, features, sparse_feature_columns, history_feature_list,
                                      history_feature_list,to_list=True)
    keys_emb_list = embedding_lookup(embedding_dict, features, history_feature_columns, history_fc_names,
                                     history_fc_names,to_list=True)
    dnn_input_emb_list = embedding_lookup(embedding_dict, features, sparse_feature_columns,
                                          mask_feat_list=history_feature_list,to_list=True)
    dense_value_list = get_dense_input(features, dense_feature_columns)

    sequence_embed_dict = varlen_embedding_lookup(embedding_dict,features,sparse_varlen_feature_columns)
    sequence_embed_list = get_varlen_pooling_list(sequence_embed_dict, features, sparse_varlen_feature_columns,to_list=True)

    dnn_input_emb_list += sequence_embed_list


    keys_emb = concat_func(keys_emb_list, mask=True)
    deep_input_emb = concat_func(dnn_input_emb_list)
    query_emb = concat_func(query_emb_list, mask=True)
    hist = AttentionSequencePoolingLayer(att_hidden_size, att_activation,
                                         weight_normalization=att_weight_normalization, supports_masking=True)([
        query_emb, keys_emb])

    deep_input_emb = Concatenate()([NoMask()(deep_input_emb), hist])
    deep_input_emb = Flatten()(deep_input_emb)
    dnn_input = combined_dnn_input([deep_input_emb],dense_value_list)
    output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn,
                 dnn_dropout, dnn_use_bn, seed)(dnn_input)
    final_logit = Dense(1, use_bias=False)(output)

    output = PredictionLayer(task)(final_logit)

    model = Model(inputs=inputs_list, outputs=output)
    return model



In [59]:
HIDDEN_SIZE = (128, 128)
BATCH_SIZE = 1024
EPOCH = 1
EMBED_DIM = 256
TIME_EMBED_DIM = 16
tf.set_random_seed(1234)
def generate_din_feature_columns(data, sparse_features, dense_features, use_time_feat=time_feat):
    sparse_feature_columns = [SparseFeat(feat, vocabulary_size=len(feat_lbe_dict[feat].classes_)+1, embedding_dim=EMBED_DIM)
                              for i, feat in enumerate(sparse_features) if feat not in time_feat]

    sparse_feature_columns += [SparseFeat(feat, vocabulary_size=time_vocab_map[feat]+1, embedding_dim=TIME_EMBED_DIM)
                              for i, feat in enumerate(use_time_feat)]
    

    dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_features]

    var_feature_columns = [VarLenSparseFeat(SparseFeat('hist_item_id', vocabulary_size=len(feat_lbe_dict['item_id'].classes_)+1,
                                                       embedding_dim=EMBED_DIM, embedding_name='item_id'), 
                                                       maxlen=max_seq_len)]

    var_feature_columns += [VarLenSparseFeat(SparseFeat('hist_{}'.format(feat), vocabulary_size=time_vocab_map[feat]+1,
                                              embedding_dim=TIME_EMBED_DIM,embedding_name=feat), maxlen=max_seq_len) 
                                                    for i, feat in enumerate(use_time_feat)]
    # DNN side
    dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns
    # FM side
    linear_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns
    # all feature names
    feature_names = get_feature_names(dnn_feature_columns+linear_feature_columns)

    return feature_names, linear_feature_columns, dnn_feature_columns

In [60]:
feature_names, linear_feature_columns, dnn_feature_columns = generate_din_feature_columns(train_final_df, ['user_id', 'item_id'], 
                                                                                          dense_features=item_dense_feat+sim_dense_feat+hist_time_diff_feat+hist_cnt_sim_feat+user_interest_dense_feat,
                                                                                          use_time_feat=[])

In [61]:
train_input = {name: np.array(train_final_df[name].values.tolist()) for name in feature_names}
train_label = train_final_df['label'].values

In [62]:
if mode == 'offline':
    val_input = {name: np.array(val_final_df[name].values.tolist()) for name in feature_names}
    val_label = val_final_df['label'].values

In [63]:
get_init_user_embed(target_phase, False)

train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=8


In [64]:
behavior_feature_list = ['item_id']
model = KDD_DIN(dnn_feature_columns, behavior_feature_list, dnn_hidden_units=HIDDEN_SIZE,
                att_hidden_size=(128, 64), att_weight_normalization=True, dnn_dropout=0.5)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=3e-4), loss="binary_crossentropy",
                  metrics=['binary_crossentropy', tf.keras.metrics.AUC()], )

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


init user embed
init item embed
init item embed


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
dim is deprecated, use axis instead


Instructions for updating:
dim is deprecated, use axis instead


In [93]:
if mode == 'offline':
    model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=EPOCH,
              verbose=1, validation_data=(val_input, val_label), ) # epoch. 0.8912 -> 0.8918 -> 0.8881
else:
    model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=EPOCH, verbose=1) # epoch. 0.8912 -> 0.8918 -> 0.8881

Train on 666885 samples, validate on 46714 samples
666885/666885 [==============================] - 263s 394us/sample - loss: 0.2023 - binary_crossentropy: 0.2023 - auc: 0.8675 - val_loss: 0.0724 - val_binary_crossentropy: 0.0724 - val_auc: 0.8573


In [65]:
model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=EPOCH, verbose=1)

689040/689040 [==============================] - 426s 619us/sample - loss: 0.1980 - binary_crossentropy: 0.1980 - auc: 0.8582


## generate recommend result

In [66]:
def get_recall_predict(infer_recall_df, phase):
    topk_fill_items  = online_top50_click if mode == 'online' else offline_top50_click
    result = get_predict(infer_recall_df, 'sim', topk_fill_items)
    result.to_csv(output_path + '/baseline_recall_v1_phase_{}.csv'.format(phase), index=False, header=None)
    return result

def get_rank_predict(dfm_infer_call_df, phase, infer_target_uids=None, rating_col='prob'):
    dfm_infer_call_df = dfm_infer_call_df.copy()
    fake_item = dfm_infer_call_df['item_id'].iloc[0]
    infer_users = set(dfm_infer_call_df['user_id'].unique())
  
    if infer_target_uids is None:
        infer_target_uids = infer_users

    for uid in infer_target_uids:
        if uid not in infer_users:
            print('uid={} not in infer_users'.format(uid))
            dfm_infer_call_df = dfm_infer_call_df.append({'user_id': uid, 'item_id': fake_item, 'prob': -10000}, ignore_index=True)

    dfm_infer_call_df['user_id'] = dfm_infer_call_df['user_id'].astype(int)
    dfm_infer_call_df['item_id'] = dfm_infer_call_df['item_id'].astype(int)
    
    topk_fill_items  = online_top50_click if mode == 'online' else offline_top50_click
    
    result = get_predict(dfm_infer_call_df, rating_col, topk_fill_items)
    result.to_csv(output_path + '/baseline_ranking_v1_phase_{}_{}.csv'.format(phase, mode), index=False, header=None)
    return dfm_infer_call_df, result

In [76]:
def infer_process(phase, load_from_file=True, is_sliding_compute_sim=True, is_use_whole_click=False, 
                         is_kmeans=False, is_w2v=True, is_interest=True, prefix='', ):
    
    all_click, target_infer_user_df = get_phase_click(phase)
    
    recall_methods={'item-cf', 'bi-graph', 'user-cf', 'swing'}
    if is_use_whole_click:
        print('use whole click')
        phase_whole_click = get_whole_phase_click(all_click, target_infer_user_df)
        infer_user_item_time_dict = get_user_item_time_dict(phase_whole_click)        
        phase_click = phase_whole_click
    else:
        infer_user_item_time_dict = get_user_item_time_dict(all_click)
        phase_click = all_click
        
    compute_mode = 'multi' if is_sliding_compute_sim else 'once'
    
    save_training_path = os.path.join(user_data_dir, 'recall', mode)
    sim_path = os.path.join(save_training_path, prefix + 'phase_{}_sim.pkl'.format(phase))
    recall_path = os.path.join(save_training_path, prefix+ 'phase_{}.pkl'.format(phase))
    
    if load_from_file:
        print('load recall info from file begin, recall_path={}'.format(recall_path))  
        full_sim_pair_dict = pickle.load(open(sim_path, 'rb'))
        infer_user_recall_item_dict = pickle.load(open(recall_path, 'rb'))
        print('load recall info from file done')  
    else:
        item_cnt_dict = all_click.groupby('item_id')['user_id'].count().to_dict()
        user_cnt_dict = all_click.groupby('user_id')['item_id'].count().to_dict()
        
        full_sim_pair_dict = get_multi_source_sim_dict_results_multi_processing(phase_click, recall_methods=recall_methods) 
        infer_user_recall_item_dict = do_multi_recall_results_multi_processing(full_sim_pair_dict, infer_user_item_time_dict, 
                                                                     target_user_ids=target_infer_user_df['user_id'].unique(), 
                                                                     ret_type='tuple', phase=phase,
                                                                     item_cnt_dict=item_cnt_dict,  user_cnt_dict=user_cnt_dict, 
                                                                     adjust_type='v2', recall_methods=recall_methods | {'sr-gnn'})
        
        pickle.dump(full_sim_pair_dict, open(sim_path, 'wb'))
        pickle.dump(infer_user_recall_item_dict, open(recall_path, 'wb'))
        
    infer_recall_recom_df = organize_recall_feat(infer_user_recall_item_dict, infer_user_item_time_dict, 
                                                                  full_sim_pair_dict, phase)
     
    target_infer_user_df['day_id'],  target_infer_user_df['hour_id'], target_infer_user_df['minute_id'] = zip(*target_infer_user_df['time'].apply(time_info))
    infer_recall_recom_df = pd.merge(infer_recall_recom_df, target_infer_user_df[['user_id', 'time', 'day_id', 'hour_id', 'minute_id']], on='user_id', how='left')


    infer_final_df = organize_user_item_feat(infer_recall_recom_df, processed_item_feat_df,
                                          sparse_feat, dense_feat, is_w2v=is_w2v, is_interest=is_interest)
  
    return infer_recall_recom_df, infer_final_df

In [78]:
today = '20200617' # time.strftime("%Y%m%d")
infer_recall_recom_df, infer_df = infer_process(target_phase, load_from_file=True, 
                                                is_sliding_compute_sim=True, is_use_whole_click=False, 
                                                is_w2v=True, is_interest=True, prefix='B-recall-{}_'.format(today))

train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=8
load recall info from file begin, recall_path=user_data/recall/online/B-recall-20200617_phase_8.pkl
load recall info from file done
origin data done
obtain user dynamic feat done
interest done
word2vec done
user_id
item_id
day_id
hour_id


In [79]:
# lgb ranking results
# lgb_infer_ans = clf.predict_proba(infer_df[lgb_cols],  axis=1)[:,1]
# infer_recall_recom_df['prob'] = lgb_infer_ans

In [ ]:
lgb_rank_infer_ans = lgb_rank.predict(infer_df[lgb_cols],  axis=1)
infer_recall_recom_df['prob'] = lgb_rank_infer_ans

In [81]:
# din ranking results
infer_input = {name: np.array(infer_df[name].values.tolist()) for name in feature_names}
din_infer_ans = model.predict(infer_input, batch_size=BATCH_SIZE)
infer_recall_recom_df['prob'] = din_infer_ans

In [82]:
recall_res = get_recall_predict(infer_recall_recom_df, target_phase)
infer_recall_df, rank_res = get_rank_predict(infer_recall_recom_df, target_phase, rating_col='prob')

90900
90900


In [84]:
# phase 8: recall results
recall_res

user_id       0       1      2      3       4       5      6       7  \
0           8   34006   53661  13933  39921   81620   13864  25392   20004   
1          30  110172   90156   5197  47210   48047   49196  28881  104786   
2          41   58934  102105  18873  63939  110613   51699  79771    4541   
3          52   87799   61755  56565   7156   27286    4639  39648   21557   
4          63    1129  109885   9483  48236   32605   81607  70502   79340   
...       ...     ...     ...    ...    ...     ...     ...    ...     ...   
1813    35296   38714   46625  14653  50018   14267  110089  99661   97394   
1814    35340   65886   42129  63614  14267   20290   34551  80109   72168   
1815    35362  102254  101886  64742  33482  111646   11232  20004     630   
1816    35395   12638   88109   2436  32500   81754   19071  40661   26577   
1817    35406   96445   18319  52222   8717    9098   40642  56916    9230   

          8  ...     40     41     42      43     44     45     46     47  \
0     40318  ...    298  35192  13948   53664  41822  84209  18107   6512   
1     71944  ...  62233  86220  86950   70151  32500  67634   4690  31171   
2     55624  ...  71408  90742  13302  106255  43663   1604   1882  17591   
3     93581  ...  16707  26312  38839   35537  26146  86510  81338  49545   
4     20463  ...  72382  29724  57911   40604  15813  65951  40535  41483   
...     ...  ...    ...    ...    ...     ...    ...    ...    ...    ...   
1813  39977  ...  98915  18347  50807   69816  69608  26572  85237  33199   
1814  21743  ...  11521  50294  92829   32604  45014  14799  19256  20642   
1815   4822  ...  93462  79081  21112   63362  83795  60007  98897  57598   
1816  32737  ...   8721   8930  46186   19881  88380  25018  94897  79043   
1817  29907  ...  33940  63982   5548   21293  20740  12953  21134  76225   

          48      49  
0        204    9879  
1      35264   34903  
2      10596   76465  
3       8117   70274  
4      14954   19925  
...      ...     ...  
1813  102119  107304  
1814   73314   94748  
1815   79347   76548  
1816   90289   93959  
1817   21721   20754  

[1818 rows x 51 columns]

In [83]:
# phase 8: rank results of din
rank_res

user_id       0       1      2       3       4      5       6      7  \
0           8   34006   53661   8563   55580   13933  13864     570  39921   
1          30  110172   76169   5197   90156    2611  48047  114108  49196   
2          41   12845    4541  18873   79771  102105  80626   13302  55624   
3          52   46738   56565  21557   39403   93581   7153   27286  39648   
4          63    1129   20463   8968  109885    9483  32605   25332  79340   
...       ...     ...     ...    ...     ...     ...    ...     ...    ...   
1813    35296   38714   60119  97394   50018   42697  84177   88984  99661   
1814    35340   42129   14267  65886   25724   34551  43835   80894  63614   
1815    35362   33482  102254    630  101886   64742  22227   36397  54848   
1816    35395    2436   57480  88109   12638   23477  37790   73514  27897   
1817    35406   34757    9098  19638   53309  113128   9230  103941  50234   

           8  ...     40      41      42      43     44     45     46     47  \
0       2773  ...  41822    5507    6230   84209  16081    204    298  81107   
1     106286  ...  35804   32500   30496   11170  59038  90006  32925  52766   
2      71408  ...  10596   24986  116327  116102   8798   1882    151   4230   
3      87654  ...   4116   34716    2602   36175  38839  63654   4230   4866   
4      70502  ...  57911   81697   15813   33847  65951  73479  14807  19925   
...      ...  ...    ...     ...     ...     ...    ...    ...    ...    ...   
1813   30973  ...  78549   10583   50807   33732  60856  76573  85518  56349   
1814   12365  ...  10225   52307   23552   65128  23026  73979  51571  34110   
1815   69371  ...  96529  111124  105924   33659  71126  63362  57598  57242   
1816   32500  ...  45471   26577    9777   15958  77877  85326   2556   7665   
1817   18319  ...  74658    8036   44987    4385   1982  48087  21293   1807   

          48     49  
0       6512    332  
1      28415  35668  
2      28505  12981  
3      38222  42651  
4      57873  14866  
...      ...    ...  
1813   10604  98915  
1814   69526   1331  
1815   60070  64746  
1816  101814  88380  
1817   26129   5435  

[1818 rows x 51 columns]

## ranking pipeline

In [87]:
# read recall results 
total_recom_lgb_df  = sub2_df(os.path.join(output_path, 'result.csv'))

In [85]:
def ranking_pipeline(target_phase, output_ranking_filename=None, model_names=['ranker'], 
                     is_train_load_from_file=True, is_infer_load_from_file=True, recall_prefix='', save_df_prefix='', feat_cols=None):
    global total_recom_lgb_df, word2vec_item_embed_dict
    
    ranking_final_data = os.path.join(user_data_dir, 'ranking')
    if not os.path.exists(ranking_final_data): os.makedirs(ranking_final_data)

    train_df_path = os.path.join(ranking_final_data, save_df_prefix + 'train_final_df_phase_{}.pkl'.format(target_phase))
    val_df_path = os.path.join(ranking_final_data, save_df_prefix + 'val_final_df_phase_{}.pkl'.format(target_phase))
    w2v_path = os.path.join(ranking_final_data, save_df_prefix + 'w2v_phase_{}.pkl'.format(target_phase))
    
    if is_train_load_from_file and os.path.exists(train_df_path):
        print('load train from file...')
        train_final_df = pickle.load(open(train_df_path, 'rb'))
        word2vec_item_embed_dict = pickle.load(open(w2v_path, 'rb'))
        if mode == 'offline':
            val_final_df = pickle.load(open(val_df_path, 'rb'))
    else:  
        if mode == 'online':
            train_full_df = online_train_full_df_dict[target_phase]
            if isinstance(train_full_df, list):
                train_full_df = train_full_df[0]
        else:
            train_full_df = train_full_df_dict[target_phase]
            val_full_df = val_full_df_dict[target_phase]
            
        word2vec_item_embed_dict = get_word2vec_feat(train_full_df)
        train_final_df = organize_user_item_feat(train_full_df, processed_user_feat_df, 
                                                    processed_item_feat_df, sparse_feat, dense_feat, is_w2v=True, is_kmeans=False)
        pickle.dump(train_final_df[use_feats + ['label']], open(train_df_path, 'wb'))
        pickle.dump(word2vec_item_embed_dict, open(w2v_path, 'wb'))
        
        if mode == 'offline':
            val_final_df = organize_user_item_feat(val_full_df, processed_user_feat_df, 
                                                   processed_item_feat_df, sparse_feat, dense_feat, is_w2v=True, is_kmeans=False)
            pickle.dump(val_final_df[use_feats + ['label']], open(val_df_path, 'wb'))
   
    print('prepare train data done...')
    
    # load infer 
    infer_df_path = save_df_prefix + recall_prefix + 'infer_final_df_phase_{}.pkl'.format(target_phase)
    
    if is_infer_load_from_file and os.path.exists(infer_df_path):
        print('load infer from file...')
        infer_recall_recom_df, infer_df = pickle.load(open(infer_df_path, 'rb'))
    else:
        train_full_df = online_train_full_df_dict[target_phase]
        if isinstance(train_full_df, list):
            train_full_df = train_full_df[0]
        infer_recall_recom_df, infer_df = infer_process(target_phase, load_from_file=True, 
                                                    is_sliding_compute_sim=True, is_use_whole_click=True, prefix=recall_prefix)
#         pickle.dump([infer_recall_recom_df, infer_df[use_feats]], open(infer_df_path, 'wb'))
    
    print('prepare infer data done...')

       
    def gen_rec_results(output_model_name):
        global total_recom_lgb_df
        rank_output_dir = os.path.join(user_data_dir, 'rank')
        if not os.path.exists(rank_output_dir): os.makedirs(rank_output_dir)
        # recall
        if mode == 'online':
            # check
            assert len(set(infer_recall_recom_df['user_id'].unique())-set(total_recom_lgb_df[total_recom_lgb_df['phase'] == target_phase].user_id.unique())) == 0# output
            total_recom_lgb_df = total_recom_lgb_df[total_recom_lgb_df['phase'] != target_phase]
            online_infer_recall_recom_df = infer_recall_recom_df[['user_id', 'item_id', 'prob']].rename(columns={'prob': 'sim'})
            online_infer_recall_recom_df['phase'] = target_phase
            total_recom_lgb_df = total_recom_lgb_df.append(online_infer_recall_recom_df)

            result = get_predict(total_recom_lgb_df, 'sim', online_top50_click)
            result.to_csv( '{}/{}-{}'.format(rank_output_dir, output_model_name, output_ranking_filename), index=False, header=None)
            pickle.dump(total_recom_lgb_df, open("{}/{}-{}-pkl".format(rank_output_dir, output_model_name, output_ranking_filename), 'wb'))
        
        print('generate rec result done...')
        
            
    if  'ranker' in model_names:
        print('ranker begin....')
        train_final_df.sort_values(by=['user_id'], inplace=True)
        g_train =  train_final_df.groupby(['user_id'], as_index=False).count()["label"].values
        if mode == 'offline':
            val_final_df.sort_values(by=['user_id'], inplace=True)
            g_val = val_final_df.groupby(['user_id'], as_index=False).count()["label"].values

        lgb_ranker = lgb.LGBMRanker(
                boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                max_depth=-1, n_estimators=300, objective='binary',
                subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs=-1) # 300epoch, best, 0.882898, dense_feat  + hist_cnt_sim_feat user_interest_dense_feat

        if mode == 'offline':
            lgb_ranker.fit(train_final_df[feat_cols],  train_final_df['label'], group=g_train, 
                         eval_set=[(val_final_df[feat_cols], val_final_df['label'])], eval_group=[g_val], 
                         eval_at=[50], eval_metric=['auc',],  
                         early_stopping_rounds=50, ) 
        else:
            lgb_ranker.fit(train_final_df[feat_cols],  train_final_df['label'], group=g_train)
        
        print('train done...')
        lgb_rank_infer_ans = lgb_ranker.predict(infer_df[feat_cols],  axis=1)
        infer_recall_recom_df['prob'] = lgb_rank_infer_ans
        gen_rec_results('ranker')
        

    if 'din' in model_names:
        print('din begin...')
        get_init_user_embed(target_phase, is_use_whole_click=True)
        feature_names, linear_feature_columns, dnn_feature_columns = generate_din_feature_columns(train_final_df, ['user_id', 'item_id'], 
                                                                                          dense_features=item_dense_feat+sim_dense_feat+hist_time_diff_feat+hist_cnt_sim_feat+user_interest_dense_feat,
                                                                                          use_time_feat=[])
        train_input = {name: np.array(train_final_df[name].values.tolist()) for name in feature_names}
        train_label = train_final_df['label'].values
        if mode == 'offline':
            val_input = {name: np.array(val_final_df[name].values.tolist()) for name in feature_names}
            val_label = val_final_df['label'].values
        
        EPOCH = 1
        behavior_feature_list = ['item_id'] 
        model = KDD_DIN(dnn_feature_columns, behavior_feature_list, dnn_hidden_units=HIDDEN_SIZE,
                    att_hidden_size=(128, 64), att_weight_normalization=True, 
                    dnn_dropout=0.5)
        
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=3e-4), loss="binary_crossentropy",
                      metrics=['binary_crossentropy', tf.keras.metrics.AUC()], )

        if mode == 'offline':
            model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=EPOCH,
                  verbose=1, validation_data=(val_input, val_label), ) # 1:20目前最优结果, epoch. 0.8728
        else:
            model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs=EPOCH,
                  verbose=1)
        infer_input = {name: np.array(infer_df[name].values.tolist()) for name in feature_names}
        din_infer_ans = model.predict(infer_input, batch_size=BATCH_SIZE)
        infer_recall_recom_df['prob'] = din_infer_ans
        gen_rec_results('din')

    

In [88]:
output_ranking_filename = "B-ranking-{}".format(today)
use_feats = ['user_id', 'item_id'] + ['hist_item_id', ]  + lgb_cols 

for i in range(start_phase, now_phase+1):
    print('phase={}'.format(i))
    output_ranking_filename = output_ranking_filename + "_" + str(i)
    ranking_pipeline(i, output_ranking_filename + '.csv', model_names=['ranker', 'din'], 
                            is_train_load_from_file=True,
                            is_infer_load_from_file=True, 
                            recall_prefix='B-recall-{}_'.format(today),
                            save_df_prefix='B-{}_'.format(today),
                            feat_cols=lgb_cols)

phase=7
load train from file...
prepare train data done...
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=7
use whole click
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742      0
20480        1    89568  0.983763      0
20968        1    97795  0.983877      0
group done
load recall info from file begin, recall_path=user_data/recall/online/B-recall-20200617_phase_7.pkl
load recall info from file done
origin data done
obtain user dynamic feat done
interest done
word2vec done
user_id
item_id
day_id
hour_id
prepare infer data done...
ranker begin....
train done...
268350
generate rec result done...
din begin...
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=7
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742  

692781/692781 [==============================] - 777s 1ms/sample - loss: 0.2059 - binary_crossentropy: 0.2059 - auc_1: 0.8500
268350
generate rec result done...
phase=8
load train from file...
prepare train data done...
train_path=data/underexpose_train, test_path=data/underexpose_test, target_phase=8
use whole click
       user_id  item_id      time  phase
3123         1    47611  0.983887      0
19709        1    76240  0.983770      0
19829        1    78142  0.983742      0
20480        1    89568  0.983763      0
20968        1    97795  0.983877      0
group done
load recall info from file begin, recall_path=user_data/recall/online/B-recall-20200617_phase_8.pkl
load recall info from file done
origin data done


KeyboardInterrupt: 

## ensemble 

In [89]:
def norm_sim(sim_df, weight=0.0):
  min_sim = sim_df.min()
  max_sim = sim_df.max()
  if max_sim == min_sim:
    sim_df = sim_df.apply(lambda sim: 1.0)
  else:
    sim_df = sim_df.apply(lambda sim: 1.0 * (sim - min_sim) / (max_sim - min_sim))
    
  sim_df = sim_df.apply(lambda sim: sim+weight) # plus one
  return sim_df

In [90]:
rank_output_dir = os.path.join(user_data_dir, 'rank')

In [78]:
# read lgb
lgb_output_file = 'ranker-' + output_ranking_filename + '.csv-pkl'
lgb_full_df = pickle.load(open('{}/{}'.format(rank_output_dir, lgb_output_file), 'rb'))
lgb_full_df['sim'] = lgb_full_df.groupby('user_id')['sim'].transform(lambda df: norm_sim(df))

In [67]:
# read din
din_output_file = 'din-' + output_ranking_filename + '.csv-pkl'
din_full_df = pickle.load(open('{}/{}'.format(rank_output_dir, din_output_file), 'rb'))
din_full_df['sim'] = din_full_df.groupby('user_id')['sim'].transform(lambda df: norm_sim(df))

In [68]:
# fuse lgb and din
din_lgb_full_df = lgb_full_df.append(din_full_df)
din_lgb_full_df = din_lgb_full_df.groupby(['user_id', 'item_id', 'phase'])['sim'].sum().reset_index()

In [79]:
# output fuse results
res3 = get_predict(din_lgb_full_df, 'sim', online_top50_click)
res3.to_csv(output_path + '/B-0615_ranker_din.csv', index=False, header=None)

268350


In [81]:
# xtf_v6_ranker
res3

user_id      0       1       2       3       4       5      6       7  \
0           7  79771  115146   10596  113031  114953  115074  13954  117549   
1           8  13933    8563   67448   34006   53661   40318  39921    6552   
2           9   7057   19724   21431   11170   12600   21504  21625   67156   
3          29  16401   16751   26442   19520   33816   25034  17213   18399   
4          30  28881   48047  106286   49196   76169   90156  47210  110172   
...       ...    ...     ...     ...     ...     ...     ...    ...     ...   
5362    35395   2436   32500   12638   88109   57319   40661  32737   23477   
5363    35406   9098   19638   52222    9230     807    8717  34757   77154   
5364    35418  69060   43776  102479   30800    7249   81447  87207   33109   
5365    35429  79490    2213   97400   34551   29322   41178   9408   37872   
5366    35440  31249   56255   19988   62387   82469    4612  46301     110   

           8  ...      40      41      42      43      44     45      46  \
0     117580  ...  103876  116327   91297  101596  114711  58934  116977   
1      41822  ...   13467   53664   24530    2786  116788    569   47163   
2      45738  ...   64919    1390   16971   70887  114477     29   33446   
3     110573  ...   14993   17952   26380   66751   88804  79860   62119   
4      24350  ...   67525   77047   37029   31171   97196  79294   16391   
...      ...  ...     ...     ...     ...     ...     ...    ...     ...   
5362   32466  ...   12071   48961   36991   72577   32602  10856    9777   
5363   34436  ...    8790   28725  113128    1807    8036  89857    3123   
5364   86554  ...   45643  107596    1648   21439   12643   6707    4698   
5365   65955  ...   94524   84470    6198   52300   87708  50374   43037   
5366   35152  ...   96300   21279    5014   99225   34716   6437   17697   

          47     48      49  
0     117577  12661   98323  
1      26770   6834   16081  
2      19435  72338   20705  
3      54968    713   67418  
4     102244   6568    2242  
...      ...    ...     ...  
5362   73514  24563   46300  
5363   49722   9662  112783  
5364   12077  91524   84957  
5365  101796  80216   92349  
5366    9356  17646  111140  

[5367 rows x 51 columns]

In [70]:
# xtf_v6_ranker_din 的结果
res3

user_id      0       1       2       3       4       5       6       7  \
0           7  79771  115146   10596  113031  114953  115074   13954  117549   
1           8  13933    8563   67448   34006   53661   40318   39921    6552   
2           9   7057   19724   21431   11170   21504   67156   12600   77133   
3          29  16401   16751   26442   19520   33816   25034   17213   18399   
4          30  28881   48047  106286   49196   76169   90156   47210  110172   
...       ...    ...     ...     ...     ...     ...     ...     ...     ...   
5362    35395   2436   32500   12638   88109   57319   40661   32737   23477   
5363    35406   9098   19638   52222    9230     807    8717   34757   77154   
5364    35418  69060   43776    7249   87207   33109  102479   20921   30800   
5365    35429  79490    2213   34551   29322   41178   51467    9408   97400   
5366    35440  31249   62387   56255   19988   82469   46301  109649   43757   

           8  ...      40      41      42      43      44     45      46  \
0     117580  ...  103876  116327   91297  101596  114711  58934  116977   
1      41822  ...   13467   53664   24530    2786  116788    569   47163   
2      21625  ...   20066   13408   21034    6398   35705   2611   78086   
3     110573  ...   14993   17952   26380   66751   88804  79860   62119   
4      24350  ...   67525   77047   37029   31171   97196  79294   16391   
...      ...  ...     ...     ...     ...     ...     ...    ...     ...   
5362   32466  ...   12071   48961   36991   72577   32602  10856    9777   
5363   34436  ...    8790   28725  113128    1807    8036  89857    3123   
5364   81447  ...   16489   51258   17513   27951   14305  59765   96092   
5365   70253  ...   94524   32176   92234   15850  114612  20670   48307   
5366   70057  ...   77877   96300   69717   45451   21175  17646   71978   

          47     48      49  
0     117577  12661   98323  
1      26770   6834   16081  
2      15959  33867   72338  
3      54968    713   67418  
4     102244   6568    2242  
...      ...    ...     ...  
5362   73514  24563   46300  
5363   49722   9662  112783  
5364   78814  12643   23215  
5365   91126  53258   70042  
5366   21426  30832     733  

[5367 rows x 51 columns]

In [66]:
# A榜的0.61对应的B榜结果
res

user_id       0       1      2       3       4      5       6       7  \
0           7   79771  113031  57786  115146  117580  87254  115074   13954   
1           8    8563   13933  67448   13864   55580  34006     570   53661   
2           9    7057   21431  11170   19724    4340  80227    3302   21504   
3          29   16401   16751  26442   45767   17213  36421   24744   18399   
4          30  110172   90156  76169   49196   48047  28881  104786  106286   
...       ...     ...     ...    ...     ...     ...    ...     ...     ...   
5362    35395   32500    2436  12638   27897   88109  57480   40661   57319   
5363    35406    9098   52222  59875    9230   34757  19638   53309   26611   
5364    35418   43776    7249  45837   69060   13063  20921   27500   12077   
5365    35429   79490   65955   2213   41178   70253  58100   51467    9408   
5366    35440   31249   56255  62387   82469   19988  76752   70057   45451   

           8  ...      40     41      42      43      44     45      46  \
0     116475  ...  117582  91297  116977  116327  114711   4340   52766   
1      40318  ...   89720   6512   25532    7765   17138  38435   27936   
2      67156  ...   35537  78086   19260   13408  103605  64919   15974   
3      14038  ...   22995  88632   80118   67418   79860  71421    1218   
4      70985  ...   79294  31434   53622   76833   32925  14879   35341   
...      ...  ...     ...    ...     ...     ...     ...    ...     ...   
5362   26022  ...   12906  19881   85326   89537   48961  64726   32816   
5363     807  ...   18442  21134   41626    8790   89857  39645   49722   
5364   86554  ...   81447  20918    4698   28322   13551  51258    1200   
5365   34551  ...   13540  84921   50374   80894   72853  99732  105070   
5366      93  ...  110089    110   57079   95211   71505  96300   88652   

          47      48      49  
0      24411   42155   71408  
1       6834   24686     569  
2      52062   55626   98736  
3      81049   56669   23874  
4       5872   47886  114108  
...      ...     ...     ...  
5362   35407   83605   72577  
5363    8036   79198  113944  
5364   27951   56823   37961  
5365  106286   20670  101796  
5366  105255  102642  112156  

[5367 rows x 51 columns]

In [72]:
# zjl recall results 
ranker_best_full_df = pickle.load(open('{}/{}'.format(output_path, 'ranker-B-0606_item_fill_double_srgnn_zjl_7_8_9.csv-pkl'), 'rb'))
ranker_best_full_df['sim'] = ranker_best_full_df.groupby('user_id')['sim'].transform(lambda df: norm_sim(df))

din_best_full_df = pickle.load(open('{}/{}'.format(output_path, 'din-B-0606_item_fill_double_srgnn_zjl_7_8_9.csv-pkl'), 'rb'))
din_best_full_df['sim'] = din_best_full_df.groupby('user_id')['sim'].transform(lambda df: norm_sim(df))

din_lgb_best_full_df = ranker_best_full_df.append(din_best_full_df)
din_lgb_best_full_df = din_lgb_best_full_df.groupby(['user_id', 'item_id', 'phase'])['sim'].sum().reset_index()

# res_zjl = get_predict(din_lgb_best_full_df, 'sim', online_top50_click)
# res_zjl.to_csv(output_path + '/B-0606_item_fill_double_srgnn_zjl.csv', index=False, header=None)

In [78]:
res_zjl

user_id       0       1       2       3       4       5      6       7  \
0           7  116475  117580  115146  115074  117518  115104  82720  114953   
1           8   13933    8563   67448   34006   39921     628   2773   53661   
2           9    7057   12600   11170   19724   21431   77133  21504   67156   
3          29   16401   16751   48074    9329    6378   19784  60886   56669   
4          30   76169   28881   48047   86313   27663   23471  47210   90156   
...       ...     ...     ...     ...     ...     ...     ...    ...     ...   
5362    35395    2436   57480   12638   44712   32466   32500  46110   88109   
5363    35406    9230   19638     807   34757    9098   56916  29907   26611   
5364    35418   69060   33109  102479   87207   84764   52670   7249   81447   
5365    35429   79490    2213    9408   51467   70253   41178  97400   79246   
5366    35440   31249   56255   19988   76752   82469   62387  70057   59549   

           8  ...      40     41      42     43      44     45      46  \
0     117545  ...  115145  48639   68115  86109  116977  42155   78607   
1      40318  ...    3258  14647    5507  31152   15029  53664   26770   
2      21625  ...   78086   3302   84131  42619   33355  80227    1735   
3      45767  ...   43847  66454   25868   2242   49506  20282   23759   
4      49196  ...   89340  97196  107416  62233   79294  67525   31171   
...      ...  ...     ...    ...     ...    ...     ...    ...     ...   
5362   14785  ...    7665  26022   89537  76480   56466  72577   24563   
5363   52222  ...    9662  79336   11007  14816    3123  89857   96445   
5364   30800  ...   59765   1709   12077  27500   78814  44755   85434   
5365   29322  ...   62337  76736   34229  92234   23205  56551   22780   
5366   46301  ...   35152  57797     110  61241   45451  21175  106286   

          47      48     49  
0     117582  117291   4604  
1      24530  116788   5965  
2      33867   65790  32959  
3      17952   46438   6272  
4      41871   26306  19517  
...      ...     ...    ...  
5362   67690   28149  60451  
5363    8036    1982   9455  
5364    3937   11224  96092  
5365   77767  114612  52410  
5366   30832    6437  23120  

[5367 rows x 51 columns]

In [73]:
# 0.61 recall results + zjl recall results
din_lgb_best_full_boost_df = din_lgb_best_full_df.append(din_lgb_full_df)
din_lgb_best_full_boost_df = din_lgb_best_full_boost_df.groupby(['user_id', 'item_id', 'phase'])['sim'].sum().reset_index()

res4 = get_predict(din_lgb_best_full_boost_df, 'sim', online_top50_click)
res4.to_csv(output_path + '/B-0606_item_fill_double_srgnn_ranker_din_boost_xtf_v6_zjl.csv', index=False, header=None)

268350


In [73]:
# fuse two times recall results, 0.6621, hitrate_50_full:1.4962, ndcg_50_full:0.6621, hitrate_50_half:1.0577, ndcg_50_half:0.4421
res2

user_id      0       1       2       3       4       5       6       7  \
0           7  79771  117580  115146  116475  115074  115104  113031  117545   
1           8  13933    8563   67448   34006   39921     570   53661   40318   
2           9   7057   21431   11170   19724   21504   67156   12600   21625   
3          29  16401   16751   26442   45767   17213   33816   18399  110573   
4          30  76169   28881   48047   90156   49196  110172   47210  106286   
...       ...    ...     ...     ...     ...     ...     ...     ...     ...   
5362    35395   2436   12638   57480   32500   88109   32466   14785   57319   
5363    35406   9230    9098   19638     807   34757   52222   26611   77154   
5364    35418  69060   43776    7249   33109   87207   20921  102479   30800   
5365    35429  79490    2213   41178   70253    9408   51467   65955   97400   
5366    35440  31249   56255   62387   19988   82469   76752   70057   46301   

           8  ...      40      41      42     43     44      45      46  \
0     114953  ...  103876  117577  116327  91297  42155   86109  115732   
1      41822  ...     411   24686     332  48857  79075   14647   13467   
2      77133  ...   78086   55626   64919   6398  33355   20705   52766   
3      19520  ...   28601     713   72815  22995  28194   71421   57243   
4     104786  ...  110968   16391   79294  72266  40772   97196   24878   
...      ...  ...     ...     ...     ...    ...    ...     ...     ...   
5362   46110  ...   32602   46186   12071  11582    514   85326   68934   
5363   29907  ...   47017   70598   49722  40465  89857   35844    7171   
5364   45837  ...   17513   85434   78814   9341  85525   14305   27951   
5365   79246  ...   50936  101796   43727  43281  42600   50374   66904   
5366   59549  ...   35152    7136   69752  92561  30832  116683  106286   

          47      48      49  
0     115145  117291  114201  
1      84209   46611    3258  
2      32959   98736   83180  
3       1218   16690   28819  
4      10566   14879   76833  
...      ...     ...     ...  
5362   29717   40284   88211  
5363    8036   41936    8790  
5364   36065   53809    4698  
5365   72853   22820   63099  
5366   55033   61241   21175  

[5367 rows x 51 columns]

In [226]:
# good din_ranker_fuse_results_references， score: 0.665/1.503/0.665/1.040/0.426
res

user_id       0       1       2       3       4       5      6      7  \
0            1   32360  100116   21808   99628  100058   64538  46297  78641   
1            2   55815   18104   22914   81139   16414   82469  51505  87032   
2            3   87420   47622   24847   48446   10716   35152  78914  37485   
3           11   26711   40801   77847   21517   46946   59376   3506  59255   
4           13  111918  112207   42109   80126  109415   89440  21422  99276   
...        ...     ...     ...     ...     ...     ...     ...    ...    ...   
12076    35426  101826    1760   89709   22636   76625   34034  17282  82951   
12077    35434   43017   70643   81022   64550   70705   66868  60349  94200   
12078    35435   31115   91026   94624   27822   10733   79138  67467  50594   
12079    35436   29411   87047   85176   15259   27099   63342   7387  41277   
12080    35437  108450  111891  112683  114076     899  110489  69019  74340   

            8  ...      40      41      42      43      44      45      46  \
0      114108  ...   82908   33422     119   42651   39800   14974   40014   
1       61873  ...  104345   21601  107524   14587   43876   86487   52079   
2       87807  ...    7156   69039   83861   22083   13302  110044   22704   
3       79868  ...    5525    2602  113564  103317   82515   19231  107291   
4        4340  ...   93311  112709  100997   84098   36617   50875    6388   
...       ...  ...     ...     ...     ...     ...     ...     ...     ...   
12076   58064  ...   93295   54002   61128   42420   20179   45761   77970   
12077   84547  ...   37665   23097   68312   78216   71163   61610   32601   
12078   59079  ...   76584   31479   36249   28185   18984   70700   16030   
12079   19970  ...   37513   76966   21110   99917   50538   13349   67567   
12080   87186  ...   47955   26134  109383   26116  109317  105332   14636   

           47     48      49  
0      114055  35750   60128  
1      104357  50407   10612  
2       12845   4340   20773  
3       22281  11489  110891  
4       65930  14543   21721  
...       ...    ...     ...  
12076   75305  81334   83715  
12077   87826  62422   33218  
12078   43362  82217   34249  
12079   86783  67609  100808  
12080   48671  54259  111995  

[12081 rows x 51 columns]

In [230]:
# good din_results_references
res

user_id       0       1       2       3       4       5       6  \
0            1   32360  100116   99628  100058   64538  114108   78641   
1            2   55815   18104   22914   81139   16414   82469   51505   
2            3   87420   47622   24847   10716   35152   48446   73088   
3           11   79868   26711   20389   40801    3506   19326   46946   
4           13   42109   89440   80126  111918  112207  109415   91525   
...        ...     ...     ...     ...     ...     ...     ...     ...   
12076    35426  101826   89709    1760   65845   91579   17282   63340   
12077    35434   43017   81022   70705   70643   66868   84547   51182   
12078    35435   31115   94624   91026   10733   79138   67467   50594   
12079    35436   29411   87047   85176   63342   27099   19970   92533   
12080    35437   87186     899  112683  114076   74340  108450  111891   

            7       8  ...      40      41     42      43      44      45  \
0       46297  102457  ...  109635  114055  69132   60128   82908  108557   
1       58191   61873  ...   50407  104345  14587  104357  100106  109379   
2       69717   78914  ...    4340   25973  22129   51524  102121  107129   
3       21517   77847  ...    5413    4484   2171  110891   79633   76943   
4       68038    4340  ...   51386   76469  57171   21721   65181   14543   
...       ...     ...  ...     ...     ...    ...     ...     ...     ...   
12076   22636   26245  ...   14038   75305  49422  104150   43192   77970   
12077   64550   94200  ...   97306   43836  32601   36073   23097   53276   
12078   59079   27822  ...   36249    2262   9911   26174   18984   43776   
12079    7387   41277  ...   49435   51039  65567   28159   99917   91377   
12080  110489   69019  ...   90142  109317  90753  114473  107836   91296   

           46      47      48      49  
0       42651   87837   12460  114278  
1       43439   80765   10612   31823  
2       12845   22704   10676   64436  
3       66955    2602   89557   79345  
4       37345  108369  103042   48926  
...       ...     ...     ...     ...  
12076   99954   83715   20179    6461  
12077   78216   33218   67536   71163  
12078   44378   25882   43362   16030  
12079  109737   21110   31701   50538  
12080  107524   95317   95426   92782  

[12081 rows x 51 columns]

In [224]:
# good ranker_results_references
res

user_id       0       1       2       3       4      5       6       7  \
0            1   32360   21808  100116   99628   36252  46297   51719   67733   
1            2   18104   55815    6859   22914   81139  52766   87032   36152   
2            3   87420   47622   48446   24847   10716  87807  110798   35152   
3           11   26711   40801   77847   21517   59376  46946   59255   10528   
4           13  111918  112207  109415   80126   42109  99276   21422   41122   
...        ...     ...     ...     ...     ...     ...    ...     ...     ...   
12076    35426    1760  101826   22636   89709   34034  82951   76625   95584   
12077    35434   43017   70643   64550   60349   81022  67607   94200   66868   
12078    35435   31115   91026   27822   94624   52071  12317   36065   59079   
12079    35436   87047   29411   85176   15259   27099  47827   63342  104795   
12080    35437  111891  108450  112683  114076  110489    899   69019   74340   

            8  ...     40      41      42      43      44     45     46  \
0       35247  ...  40014   67618   82908   42651   97148  35217  95676   
1       67536  ...  55238    4612    4627   80066  104345  95025  67023   
2       37485  ...  25973   93433   22083   18404   42389  20773   5251   
3        3506  ...  69791   45652    2602   75400   94411  48468  67496   
4       69008  ...  51345   26281   30608   45416  112709  15869   5723   
...       ...  ...    ...     ...     ...     ...     ...    ...    ...   
12076   58064  ...  43724   26804   28793   13812   77970  74925  32526   
12077   70705  ...  19400   23097   25282   54859   78216  59311  70456   
12078    1498  ...  64520   70422   82986  104846   14197  70700  97624   
12079   63103  ...  32335   25044   50538   99917   30707  67609  32163   
12080  109379  ...   8885  114473  111995   66681  112403  85425  62545   

           47      48      49  
0         291   14974   36992  
1       26953   85011   24935  
2       52766   74756   29956  
3       24148   79345  113564  
4       52844   14543   47236  
...       ...     ...     ...  
12076   17596   83715   26830  
12077   71163   78329   75830  
12078    9869   91665   99193  
12079   48261  102920   67966  
12080  107748   98985   39519  

[12081 rows x 51 columns]

## Evaluation

In [ ]:
# coding=utf-8
from __future__ import division
from __future__ import print_function

import datetime
import json
import sys
import time
from collections import defaultdict

import numpy as np

# the higher scores, the better performance
def evaluate_each_phase(predictions, answers):
    list_item_degress = []
    for user_id in answers:
        item_id, item_degree = answers[user_id]
        list_item_degress.append(item_degree)
    list_item_degress.sort()
    median_item_degree = list_item_degress[len(list_item_degress) // 2]

    num_cases_full = 0.0
    ndcg_50_full = 0.0
    ndcg_50_half = 0.0
    num_cases_half = 0.0
    hitrate_50_full = 0.0
    hitrate_50_half = 0.0
    for user_id in answers:
        item_id, item_degree = answers[user_id]
        rank = 0
        while rank < 50 and predictions[user_id][rank] != item_id:
            rank += 1
        num_cases_full += 1.0
        if rank < 50:
            ndcg_50_full += 1.0 / np.log2(rank + 2.0)
            hitrate_50_full += 1.0
        if item_degree <= median_item_degree:
            num_cases_half += 1.0
            if rank < 50:
                ndcg_50_half += 1.0 / np.log2(rank + 2.0)
                hitrate_50_half += 1.0
    ndcg_50_full /= num_cases_full
    hitrate_50_full /= num_cases_full
    ndcg_50_half /= num_cases_half
    hitrate_50_half /= num_cases_half
    return np.array([ndcg_50_full, ndcg_50_half,
                     hitrate_50_full, hitrate_50_half], dtype=np.float32)

# submit_fname is the path to the file submitted by the participants.
# debias_track_answer.csv is the standard answer, which is not released.
def evaluate(submit_fname,
             answer_fname='debias_track_answer.csv', current_time=None):
    schedule_in_unix_time = [
        0,  # ........ 1970-01-01 08:00:00 (T=0)
        1586534399,  # 2020-04-10 23:59:59 (T=1)
        1587139199,  # 2020-04-17 23:59:59 (T=2)
        1587743999,  # 2020-04-24 23:59:59 (T=3)
        1588348799,  # 2020-05-01 23:59:59 (T=4)
        1588953599,  # 2020-05-08 23:59:59 (T=5)
        1589558399,  # 2020-05-15 23:59:59 (T=6)
        1590163199,  # 2020-05-22 23:59:59 (T=7)
        1590767999,  # 2020-05-29 23:59:59 (T=8)
        1591372799  # .2020-06-05 23:59:59 (T=9)
    ]
    assert len(schedule_in_unix_time) == 10
    for i in range(1, len(schedule_in_unix_time) - 1):
        # 604800 == one week
        assert schedule_in_unix_time[i] + 604800 == schedule_in_unix_time[i + 1]

    if current_time is None:
        current_time = int(time.time())
    print('current_time:', current_time)
    print('date_time:', datetime.datetime.fromtimestamp(current_time))
#     current_phase = 0
#     while (current_phase < 9) and (
#             current_time > schedule_in_unix_time[current_phase + 1]):
#         current_phase += 1
#     print('current_phase:', current_phase)
    current_phase = 6
  
    try:
        answers = [{} for _ in range(10)]
        with open(answer_fname, 'r') as fin:
            for line in fin:
                line = [int(x) for x in line.split(',')]
                phase_id, user_id, item_id, item_degree = line
                if mode == 'online':
                  assert user_id % 11 == phase_id
                # exactly one test case for each user_id
                answers[phase_id][user_id] = (item_id, item_degree)
    except Exception as e:
        return print('server-side error: answer file incorrect', e)

    try:
        predictions = {}
        with open(submit_fname, 'r') as fin:
            for line in fin:
                line = line.strip()
                if line == '':
                    continue
                line = line.split(',')
                user_id = int(line[0])
                if user_id in predictions:
                    return print('submitted duplicate user_ids')
                item_ids = [int(i) for i in line[1:]]
                if len(item_ids) != 50:
                    return print('each row need have 50 items')
                if len(set(item_ids)) != 50:
                    return print('each row need have 50 DISTINCT items')
                predictions[user_id] = item_ids
    except Exception as e:
        return print('submission not in correct format,e={}'.format(e))

    scores = np.zeros(4, dtype=np.float32)

    # The final winning teams will be decided based on phase T=7,8,9 only.
    # We thus fix the scores to 1.0 for phase 0,1,2,...,6 at the final stage.
    if current_phase >= 7:  # if at the final stage, i.e., T=7,8,9
        scores += 7.0  # then fix the scores to 1.0 for phase 0,1,2,...,6
    phase_beg = (7 if (current_phase >= 7) else 0)
    phase_end = current_phase + 1
    for phase_id in range(phase_beg, phase_end):
        for user_id in answers[phase_id]:
            if user_id not in predictions:
                return print('user_id %d of phase %d not in submission' % (
                        user_id, phase_id))
        try:
            # We sum the scores from all the phases, instead of averaging them.
            phase_score = evaluate_each_phase(predictions, answers[phase_id])
            print('phase_id={}, score={}'.format(phase_id, phase_score))
            scores += phase_score
        except Exception as e:
            return print('error occurred during evaluation, e={}'.format(e))
    
    print("score={},\nhitrate_50_full={},\nndcg_50_full={},\nhitrate_50_half={}, \nndcg_50_half={}".format(
        float(scores[0]), float(scores[2]), float(scores[0]), float(scores[3]), float(scores[1])
    ))
    return scores[0]
    # return report_score(
    #     stdout, score=float(scores[0]),
    #     ndcg_50_full=float(scores[0]), ndcg_50_half=float(scores[1]),
    #     hitrate_50_full=float(scores[2]), hitrate_50_half=float(scores[3]))

# FYI. You can create a fake answer file for validation based on this. For example,
# you can mask the latest ONE click made by each user in underexpose_test_click-T.csv,
# and use those masked clicks to create your own validation set, i.e.,
# a fake underexpose_test_qtime_with_answer-T.csv for validation.
def _create_answer_file_for_evaluation(output_answer_fname='debias_track_answer.csv'):
    train = train_path + '/underexpose_train_click-%d.csv'
    test = test_path + '/underexpose_test_click-%d.csv'

    # underexpose_test_qtime-T.csv contains only <user_id, time>
    # underexpose_test_qtime_with_answer-T.csv contains <user_id, item_id, time>
    answer = offline_answer_path + '/underexpose_test_qtime_with_answer-%d.csv'  # not released

    item_deg = defaultdict(lambda: 0)
    with open(output_answer_fname, 'w') as fout:
        for phase_id in range(now_phase+1):
            with open(train % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    item_deg[item_id] += 1
            with open(test % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    item_deg[item_id] += 1
            with open(answer % phase_id) as fin:
                for line in fin:
                    user_id, item_id, timestamp = line.split(',')
                    user_id, item_id, timestamp = (
                        int(user_id), int(item_id), float(timestamp))
                    if mode == 'online':
                       assert user_id % 11 == phase_id
                    print(phase_id, user_id, item_id, item_deg[item_id],
                          sep=',', file=fout)
                    

# submit_fname is the path to the file submitted by the participants.
# debias_track_answer.csv is the standard answer, which is not released.
def evaluate_target_phase(submit_fname, target_phase, 
             answer_fname='debias_track_answer.csv', current_time=None):
    schedule_in_unix_time = [
        0,  # ........ 1970-01-01 08:00:00 (T=0)
        1586534399,  # 2020-04-10 23:59:59 (T=1)
        1587139199,  # 2020-04-17 23:59:59 (T=2)
        1587743999,  # 2020-04-24 23:59:59 (T=3)
        1588348799,  # 2020-05-01 23:59:59 (T=4)
        1588953599,  # 2020-05-08 23:59:59 (T=5)
        1589558399,  # 2020-05-15 23:59:59 (T=6)
        1590163199,  # 2020-05-22 23:59:59 (T=7)
        1590767999,  # 2020-05-29 23:59:59 (T=8)
        1591372799  # .2020-06-05 23:59:59 (T=9)
    ]
    assert len(schedule_in_unix_time) == 10
    for i in range(1, len(schedule_in_unix_time) - 1):
        # 604800 == one week
        assert schedule_in_unix_time[i] + 604800 == schedule_in_unix_time[i + 1]

    if current_time is None:
        current_time = int(time.time())
    print('current_time:', current_time)
    print('date_time:', datetime.datetime.fromtimestamp(current_time))
    current_phase = 0
    while (current_phase < 9) and (
            current_time > schedule_in_unix_time[current_phase + 1]):
        current_phase += 1
    print('current_phase:', current_phase)
  
    try:
        answers = [{} for _ in range(10)]
        with open(answer_fname, 'r') as fin:
            for line in fin:
                line = [int(x) for x in line.split(',')]
                phase_id, user_id, item_id, item_degree = line
                # assert user_id % 11 == phase_id
                # exactly one test case for each user_id
                answers[phase_id][user_id] = (item_id, item_degree)
    except Exception as e:
        return print('server-side error: answer file incorrect', e)

    try:
        predictions = {}
        with open(submit_fname, 'r') as fin:
            for line in fin:
                line = line.strip()
                if line == '':
                    continue
                line = line.split(',')
                user_id = int(line[0])
                if user_id in predictions:
                    return print('submitted duplicate user_ids')
                item_ids = [int(i) for i in line[1:]]
                if len(item_ids) != 50:
                    return print('each row need have 50 items')
                if len(set(item_ids)) != 50:
                    return print('each row need have 50 DISTINCT items')
                predictions[user_id] = item_ids
    except Exception as e:
        return print('submission not in correct format,e={}'.format(e))

    scores = np.zeros(4, dtype=np.float32)

    # The final winning teams will be decided based on phase T=7,8,9 only.
    # We thus fix the scores to 1.0 for phase 0,1,2,...,6 at the final stage.
    if current_phase >= 7:  # if at the final stage, i.e., T=7,8,9
        scores += 7.0  # then fix the scores to 1.0 for phase 0,1,2,...,6
    phase_beg = (7 if (current_phase >= 7) else 0)
    phase_end = current_phase + 1
    for phase_id in [target_phase]:
        for user_id in answers[phase_id]:
            if user_id not in predictions:
                return print('user_id %d of phase %d not in submission' % (
                        user_id, phase_id))
        try:
            # We sum the scores from all the phases, instead of averaging them.
            phase_score = evaluate_each_phase(predictions, answers[phase_id])
            print('phase_id={}, score={}'.format(phase_id, phase_score))
            scores += phase_score
        except Exception as e:
            return print('error occurred during evaluation e={}'.format(e))
    
    print("score={},\nhitrate_50_full={},\nndcg_50_full={},\nhitrate_50_half={}, \nndcg_50_half={}".format(
        float(scores[0]), float(scores[2]), float(scores[0]), float(scores[3]), float(scores[1])
    ))
    return scores[0]
    # return report_score(
    #     stdout, score=float(scores[0]),
    #     ndcg_50_full=float(scores[0]), ndcg_50_half=float(scores[1]),
    #     hitrate_50_full=float(scores[2]), hitrate_50_half=float(scores[3]))



In [ ]:
# read answers(val data) in offline_answer_path, and output to output_answer_fname
_create_answer_file_for_evaluation(output_answer_fname=output_path +'/debias_track_answer.csv')

In [ ]:
evaluate(submit_fname=output_path + "/baseline_cf_v1.csv", 
         answer_fname=output_path +'/debias_track_answer.csv') # itemcf

In [ ]:
evaluate_target_phase(output_path + "/baseline_recall_v1_phase_5.csv", 5, 
         answer_fname=output_path +'/debias_track_answer.csv')